In [1]:
# Environment hotfix (must run before any pandas import): block/eradicate pyarrow to avoid NumPy 2.x ABI crash
import sys, subprocess, os, importlib, importlib.abc

# 1) Proactively block any import of pyarrow (pandas will gracefully fallback if ImportError occurs)
class _BlockPyArrow(importlib.abc.MetaPathFinder):
    def find_spec(self, fullname, path=None, target=None):
        if fullname.startswith('pyarrow'):
            raise ModuleNotFoundError('Blocked pyarrow due to incompatibility with NumPy 2.x')
        return None
if not any(isinstance(f, _BlockPyArrow) for f in sys.meta_path):
    sys.meta_path.insert(0, _BlockPyArrow())

# 2) Best-effort uninstall of any existing pyarrow (safe if already absent)
try:
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'pyarrow'], check=False)
    print('Attempted pyarrow uninstall (safe if already absent).')
except Exception as e:
    print('pyarrow uninstall attempt failed (continuing):', repr(e))

# 3) Verify pandas import now works without attempting to use pyarrow
try:
    import pandas as _pd
    print('pandas imported successfully:', _pd.__version__)
except Exception as e:
    # Last-resort compatibility fix: downgrade NumPy to <2 if environment is locked with a broken pyarrow
    print('ERROR: pandas failed to import (likely due to binary pyarrow). Downgrading NumPy to <2 as fallback...')
    subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-input', 'numpy<2'], check=False)
    print('Installed numpy<2; please Restart Kernel and Run All again if import still fails.')
    raise

print('Hotfix complete. Proceeding to GPU warmup...')

# GPU Warmup & Environment Probe (runs before everything; safe no-op on CPU)
from pathlib import Path
import torch, timm

# Pin cache dirs for deterministic downloads across environments
os.environ.setdefault('TORCH_HOME', str(Path.cwd() / '.cache' / 'torch'))
os.environ.setdefault('TIMM_HOME', str(Path.cwd() / '.cache' / 'timm'))
os.environ.setdefault('HF_HOME', str(Path.cwd() / '.cache' / 'hf'))

cuda_ok = torch.cuda.is_available()
print('GPU Warmup (pre-training) | CUDA available:', cuda_ok)
if not cuda_ok:
    print('Warmup: CUDA unavailable in this runtime. This is a no-op. Proceed to GPU migration and Run All.')
else:
    dev = torch.device('cuda')
    backbones = [
        ('convnext_base', 384),
        ('tf_efficientnetv2_m_in21ft1k', 384),
    ]
    for model_name, img_size in backbones:
        try:
            print(f'[Warmup] Creating {model_name} ...')
            m = timm.create_model(model_name, pretrained=True, num_classes=120).to(dev).eval()
            xb = torch.randn(2, 3, img_size, img_size, device=dev)
            with torch.cuda.amp.autocast(True):
                _ = m(xb)
            del m, xb
            torch.cuda.synchronize()
            print(f'[Warmup] {model_name} OK.')
        except Exception as e:
            print(f'[Warmup] {model_name} FAILED:', e)
    print('Pre-training GPU warmup complete.')


Found existing installation: pyarrow 15.0.2


Uninstalling pyarrow-15.0.2:
Attempted pyarrow uninstall (safe if already absent).


ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.11/shutil.py", line 824, in move
    os.rename(src, real_dst)
OSError: [Errno 18] Invalid cross-device link: '/usr/local/lib/python3.11/dist-packages/pyarrow-15.0.2.dist-info/' -> '/tmp/pip-uninstall-uyk9szt7'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/base_command.py", line 165, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/commands/uninstall.py", line 97, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/req/req_install.py", line 638, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/lib/python3/dist-packages/pip/_internal/req/req_uninstall.py", line 369, in remove
    moved.stash(path)
 

pandas imported successfully: 2.2.2
Hotfix complete. Proceeding to GPU warmup...


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU Warmup (pre-training) | CUDA available: False
Warmup: CUDA unavailable in this runtime. This is a no-op. Proceed to GPU migration and Run All.


# Dog Breed Identification — Experiment Log and Gold-Oriented Plan (Revised)

Objective: Achieve a medal with a gold push (log loss ≤ 0.00050) on the MLE-Benchmark variant of Kaggle's Dog Breed Identification. Deliver a valid `submission.csv`.

Protocol & Hygiene:
- Concise, documented notebook; remove stale code after documenting decisions.
- Checkpoints with audits: planning (now), data audit/EDA, baseline CV results, calibration + TTA + NN step, ensemble v1, pseudo-labeling round, final.
- Programmatic notebook backups before major refactors.

Data in CWD: `train/` images, `test/` images, `labels.csv`, `sample_submission.csv`, `description.md`.

Metric: Multi-class log loss across 120 breeds.

I. Leak Detection & Data Forensics (Non-negotiable, before EDA)
- Validate all images: open errors, EXIF orientation normalization, enforce RGB, consistent dtype/range.
- Exact/near-duplicate detection within train and between train/test:
  - Perceptual hashing (e.g., phash/dhash) + Hamming thresholding for near-dupes.
  - Embedding-based similarity (pretrained robust encoder) with FAISS; cosine sim thresholds.
- If near-dupes exist, use GroupKFold keyed by duplicate groups to prevent fold leakage.
- Persist artifacts: `image_validation_report.json`, `dup_groups.json`, `train_test_sim_hits.csv`.

II. Reproducible CV Protocol
- Seeds: set `SEED=20250810` for `random`, `numpy`, `torch` (CPU/GPU), `cudnn` deterministic where viable.
- Folds: Stratified 5-fold (or GroupKFold if dup groups present). Save `fold_assignments.csv`.
- Artifact discipline per fold/model: save best checkpoint (EMA weights), training logs, OOF logits/probs, and test logits/probs pre-calibration.
- Model selection criterion: minimum validation log loss on EMA weights.

III. Baseline-to-Strong Model (to secure bronze/silver quickly)
- Backbone: `tf_efficientnetv2_m_in21ft1k` or `convnext_base` at 384.
- Loss/regularization: SoftTargetCrossEntropy; mixup≈0.2, cutmix≈0.1, label smoothing≈0.05; RandAugment; RandomErasing p≈0.25; EMA 0.9997–0.9999.
- Optimizer/schedule: AdamW (lr≈1e-3, wd≈0.02), cosine decay, warmup 1–2 epochs; AMP.
- Training: 15–20 epochs; early stopping on val log loss.
- Inference: TTA (hflip + multi-scale 384/448) and softmax probs; save OOF/test logits.

IV. Advanced Techniques for Gold Push
- Calibration (non-negotiable): Temperature scaling on stacked OOF logits (per-model and/or post-ensemble). Apply calibrated temperatures at test-time before softmax. Persist `temperatures.json`.
- Progressive resizing: staged training 256 → 384 → 512(+). Final fine-tune at higher res with low LR.
- Optimizers: investigate SAM (with base optimizer AdamW) vs AdamW; monitor calibration impact; keep EMA.
- Aggressive models & ensemble diversity:
  - Backbones: ConvNeXt (base/large/xlarge), EfficientNetV2 (M/L/XL), ViT-B/L (384–512), Swin-L.
  - 5 folds × multiple backbones/sizes; snapshot ensembling near convergence to boost diversity.
  - Resolutions: 384/448/512/600–768 as VRAM allows. Mixed-scale TTA.
  - Ensemble aggregation: geometric mean over folds × TTA × models; track both logits and probs.
- Nearest-neighbor duplicate/NN exploitation:
  - Build embeddings for train/test (strong encoder penultimate features) and index with FAISS.
  - If max cosine sim > 0.995 to a train image, hard override: set that class p≈0.999, distribute 0.001 across others (min floor 1e-6), respecting normalization.
  - For 0.985–0.995, blend: e.g., 0.8 NN-kNN vote + 0.2 ensemble probs; try k=3–10 weighted by similarity.
  - Recompute after pseudo-label fine-tunes if encoder changes.
- Pseudo-labeling round:
  - From Ensemble v1, add test samples with max prob ≥ 0.98 as pseudo-labels with sample weight 0.2–0.3.
  - Fine-tune 2–3 epochs at low LR (≈1e-4), recalibrate temperatures, re-run NN blending.
- Targeted EDA from OOF: compute per-class log loss and confusion matrix; inspect common confusions (e.g., Norfolk vs Norwich Terriers) to adjust augmentations, resolution, and class-specific sampling if needed.

V. EDA & Data Loading (post-forensics)
- Verify counts, class list (120 breeds), label distribution; check sample submission schema.
- Visual sanity checks of random samples per class.

VI. Deliverables & Artifacts
- `fold_assignments.csv`, `oof_logits.npy`, `oof_probs.npy`, per-fold checkpoints (EMA), `temperatures.json`.
- Embeddings: `train_embeds.npy`, `test_embeds.npy`, FAISS index files, `nn_hits.csv`.
- Calibration/ensemble scripts; final `submission.csv` with probabilities summing to 1, no NaNs/Inf.

Checkpoints for audit:
1) Revised plan (this cell) — includes leak detection, CV reproducibility, calibration, progressive resizing, SAM, NN duplicate exploitation, pseudo-labeling, and ensemble strategy.
2) Data forensics + EDA complete; fold assignments saved.
3) Baseline 5-fold model trained with OOF/test logits; initial TTA; bronze attempt.
4) Calibration + NN duplicate pipeline applied; silver+ attempt.
5) Ensemble v1 (≥3 backbones) + recalibration; targeted confusion analysis.
6) Pseudo-label round + final calibration/NN; gold push; finalize submission.

Risks & Mitigations:
- Overfitting: mixup/cutmix, label smoothing, EMA, SAM, strong CV.
- Leakage: duplicate/near-duplicate detection and GroupKFold.
- Calibration drift: always fit on OOF; validate on folds; avoid arbitrary sharpening.
- Runtime: prioritize one strong 5-fold model → calibration → NN step; then add 2nd/3rd backbone.

Next: Submit this revised plan for audit; upon approval, implement data forensics (validation + duplicates + embeddings) and EDA.

In [2]:
# Utility Library (Placed after Plan): Shared helpers consolidated for linear execution
# Provides: extract_embeds_dl (idempotent, cache-aware), softmax_np, row_normalize,
# log_loss_np, fit_temperature (logits), fit_temperature_from_probs (probs)

import os, math, json, time
from pathlib import Path
from typing import List, Tuple
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as TVT
from PIL import Image, ImageOps
import timm

SEED = 20250810
torch.manual_seed(SEED)
np.random.seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ---------- Generic math helpers ----------
def softmax_np(x: np.ndarray) -> np.ndarray:
    m = x.max(axis=1, keepdims=True)
    ex = np.exp(x - m)
    return ex / ex.sum(axis=1, keepdims=True)

def row_normalize(p: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    p = np.clip(p, eps, 1.0)
    p /= p.sum(axis=1, keepdims=True)
    return p

def log_loss_np(y_true: np.ndarray, probs: np.ndarray) -> float:
    p = probs[np.arange(len(y_true)), y_true]
    return float(-(np.log(np.clip(p, 1e-15, 1.0))).mean())

def fit_temperature(logits: np.ndarray, labels: np.ndarray) -> float:
    T = torch.tensor(1.0, requires_grad=True, device=device)
    x = torch.from_numpy(logits).to(device)
    y = torch.from_numpy(labels).long().to(device)
    nll = nn.CrossEntropyLoss()
    opt = torch.optim.LBFGS([T], lr=0.1, max_iter=100, line_search_fn='strong_wolfe')
    def closure():
        opt.zero_grad()
        loss = nll(x / T.clamp_min(1e-3), y)
        loss.backward()
        return loss
    opt.step(closure)
    return float(T.detach().cpu().clamp_min(1e-3).item())

def fit_temperature_from_probs(P: np.ndarray, y_true: np.ndarray, dev: str | None = None) -> float:
    dev = device if dev is None else dev
    x = torch.from_numpy(np.log(np.clip(P, 1e-12, 1.0))).to(dev)
    y_t = torch.from_numpy(y_true).long().to(dev)
    T = torch.tensor(1.0, dtype=torch.float32, requires_grad=True, device=dev)
    nll = nn.CrossEntropyLoss()
    opt = torch.optim.LBFGS([T], lr=0.5, max_iter=100, line_search_fn='strong_wolfe')
    def closure():
        opt.zero_grad()
        logits = x / torch.clamp(T, min=1e-3)
        loss = nll(logits, y_t)
        loss.backward()
        return loss
    opt.step(closure)
    return float(T.detach().cpu().clamp_min(1e-3).item())

# ---------- Minimal, cache-aware embedding extractor for forensics and kNN ----------
class _ImgOrCacheDataset(Dataset):
    def __init__(self, ids: List[str], split: str, cache_root: Path = Path('cache_384_pt'), img_size: int = 224):
        self.ids = ids
        self.split = split
        self.img_size = int(img_size)
        self.cache_dir = (cache_root / ('train' if split=='train' else 'test'))
        self.img_dir = Path('train' if split=='train' else 'test')
        # Normalize for ImageNet-pretrained models
        self.tf_img = TVT.Compose([
            TVT.Resize((self.img_size, self.img_size), interpolation=TVT.InterpolationMode.BICUBIC),
            TVT.ToTensor(),
            TVT.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ])
        self.use_cache = self.cache_dir.exists()
        self.mean = torch.tensor((0.485,0.456,0.406)).view(3,1,1)
        self.std  = torch.tensor((0.229,0.224,0.225)).view(3,1,1)
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx: int):
        id_ = self.ids[idx]
        cache_path = self.cache_dir / f"{id_}.pt"
        if self.use_cache and cache_path.exists():
            x = torch.load(cache_path, map_location='cpu', weights_only=True)
            if not isinstance(x, torch.Tensor):
                x = torch.tensor(x)
            x = x.float()
            # Resize cached tensor to requested img_size if needed
            if x.ndim == 3 and (x.shape[-2] != self.img_size or x.shape[-1] != self.img_size):
                x = F.interpolate(x.unsqueeze(0), size=(self.img_size, self.img_size), mode='bilinear', align_corners=False).squeeze(0)
            # Normalize: assume cache in [0,1]; if already normalized (values ~[-2,2]), skip
            x_mean = float(x.mean())
            x_std = float(x.std()) + 1e-12
            if not (-1.5 < x_mean < 1.5 and x_std > 0.3):
                x = (x - self.mean) / self.std
            return x, id_
        # Fallback: load from image
        img = Image.open(self.img_dir / f"{id_}.jpg")
        img = ImageOps.exif_transpose(img)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        x = self.tf_img(img)
        img.close()
        return x, id_

@torch.no_grad()
def extract_embeds_dl(ids: List[str], split: str, model_name: str = 'vit_base_patch16_224',
                      batch_size: int = 128, num_workers: int = 0, img_size: int = 224) -> np.ndarray:
    model = timm.create_model(model_name, pretrained=True, num_classes=0, global_pool='avg').to(device).eval()
    ds = _ImgOrCacheDataset(ids, split=split, cache_root=Path('cache_384_pt'), img_size=img_size)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    feats = []
    for xb, _ids in dl:
        xb = xb.to(device, non_blocking=True)
        # CPU only in current environment; AMP disabled
        f = model(xb)
        feats.append(f.float().cpu().numpy())
    embs = np.concatenate(feats, axis=0)
    # L2-normalize so IP == cosine
    embs = embs.astype(np.float32)
    embs /= (np.linalg.norm(embs, axis=1, keepdims=True) + 1e-12)
    return embs

print('Utility library loaded: helpers and extract_embeds_dl are now available for all subsequent cells.')

Utility library loaded: helpers and extract_embeds_dl are now available for all subsequent cells.


In [3]:
# Advanced Forensics: phash Hamming near-dupes (high-recall) + Embedding (FAISS) similarity with safeguards + Union-Find grouping
# Guard: If canonical artifacts already exist, skip recomputation to keep Run-All fast and deterministic on CPU-only.

from pathlib import Path
if Path('train_groups.csv').exists() and Path('train_test_sim_hits.csv').exists() and Path('nn_hits.csv').exists():
    print('Forensics artifacts detected (train_groups.csv, train_test_sim_hits.csv, nn_hits.csv). Skipping recomputation.')
else:
    # Remediations:
    # - Increase FAISS K to ensure high recall (float K=100; binary Kb=512)
    # - Split any mixed-label union-find components into breed-pure subgroups (0 mixed groups)
    # - Assert zero mixed-label groups and report final group stats
    # - SAFETY: Use cached embeddings if present; otherwise use high-throughput extract_embeds_dl (DataLoader-based) from the Utility Library (Cell 2)

    import os, sys, math, json, time, hashlib, random
    from pathlib import Path
    from collections import defaultdict
    import numpy as np
    import pandas as pd
    from PIL import Image, ImageOps

    import subprocess
    def _pip_install(pkg):
        try:
            import importlib; importlib.import_module(pkg.split('==')[0])
            return True
        except Exception:
            code = subprocess.call([sys.executable, '-m', 'pip', 'install', '--no-input', pkg])
            return code == 0

    # Ensure dependencies
    _pip_install('timm')
    _pip_install('faiss-cpu')

    import torch
    import timm
    from torchvision import transforms
    import faiss

    # Reproducibility
    SEED = 20250810
    random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    BASE = Path('.')
    TRAIN_DIR = BASE / 'train'
    TEST_DIR = BASE / 'test'

    # Load prior artifacts/metadata
    labels_df = pd.read_csv('labels.csv')
    train_meta = pd.read_csv('train_image_meta.csv')
    test_meta = pd.read_csv('test_image_meta.csv')
    sample_df = pd.read_csv('sample_submission.csv')
    classes = [c for c in sample_df.columns if c != 'id']
    breed_to_idx = {b:i for i,b in enumerate(classes)}
    id_to_breed = dict(zip(labels_df['id'], labels_df['breed']))

    # 1) High-recall phash Hamming near-duplicate detection via FAISS binary exhaustive search
    def phash_hex_to_bytes(h):
        try:
            if isinstance(h, str) and len(h) == 16:
                return bytes.fromhex(h)
        except Exception:
            pass
        return None

    train_meta['phash_bytes'] = train_meta['phash'].apply(phash_hex_to_bytes)
    test_meta['phash_bytes']  = test_meta['phash'].apply(phash_hex_to_bytes)

    def build_binary_matrix(byte_list):
        if len(byte_list) == 0:
            return np.zeros((0,8), dtype='uint8')
        return np.frombuffer(b''.join(byte_list), dtype='uint8').reshape(-1, 8)

    # Indices of samples with valid phash bytes
    train_idx_valid = [i for i, b in enumerate(train_meta['phash_bytes'].tolist()) if isinstance(b, (bytes, bytearray)) and len(b) == 8]
    test_idx_valid  = [i for i, b in enumerate(test_meta['phash_bytes'].tolist()) if isinstance(b, (bytes, bytearray)) and len(b) == 8]
    train_bin = build_binary_matrix([train_meta.at[i, 'phash_bytes'] for i in train_idx_valid])
    test_bin  = build_binary_matrix([test_meta.at[i, 'phash_bytes'] for i in test_idx_valid])

    PHASH_HAM_THR = 4
    PHASH_CORRO_THR = 2  # stronger corroboration threshold

    phash_train_pairs = []  # list of (global_train_i, global_train_j, ham)
    phash_corrob_low = set()  # pairs with ham <= 2 for FAISS corroboration
    phash_tt_hits = []        # test-train hits dicts

    if train_bin.shape[0] > 0:
        bindex = faiss.IndexBinaryFlat(64)
        bindex.add(train_bin)
        # exhaustive search; set Kb conservatively high for recall
        Kb = min(512, train_bin.shape[0])
        D, I = bindex.search(train_bin, Kb)  # Hamming distances
        seen_pairs = set()
        for qi in range(train_bin.shape[0]):
            gi = train_idx_valid[qi]
            for rk in range(1, Kb):  # skip self at rank 0
                ham = int(D[qi, rk])
                if ham > PHASH_HAM_THR:
                    continue
                gj = train_idx_valid[int(I[qi, rk])]
                a, b = (gi, gj) if gi < gj else (gj, gi)
                if (a, b) in seen_pairs:
                    continue
                seen_pairs.add((a, b))
                phash_train_pairs.append((a, b, ham))
                if ham <= PHASH_CORRO_THR:
                    phash_corrob_low.add((a, b))
    if test_bin.shape[0] > 0 and train_bin.shape[0] > 0:
        # Reuse the same train binary index
        Kb_tt = min(512, train_bin.shape[0])
        Dtt, Itt = bindex.search(test_bin, Kb_tt)
        for qi in range(test_bin.shape[0]):
            test_global = test_idx_valid[qi]
            for rk in range(Kb_tt):
                ham = int(Dtt[qi, rk])
                if ham <= PHASH_HAM_THR:
                    train_global = train_idx_valid[int(Itt[qi, rk])]
                    phash_tt_hits.append({
                        'test_id': test_meta.at[test_global, 'id'],
                        'train_id': train_meta.at[train_global, 'id'],
                        'type': 'phash_ham',
                        'distance': ham,
                    })

    print(f"phash intra-train near-dup pairs (<= {PHASH_HAM_THR}): {len(phash_train_pairs)} | corroboration (<= {PHASH_CORRO_THR}): {len(phash_corrob_low)}")
    print(f"phash train-test near hits (<= {PHASH_HAM_THR}): {len(phash_tt_hits)}")

    # 2) Embedding-based similarity with FAISS (+ safeguards for unions) — SAFETY-REFAC
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Load cached embeddings if present; otherwise compute via extract_embeds_dl (Utility Library, Cell 2) over cached tensors
    train_embeds_path = Path('train_embeds.npy')
    test_embeds_path = Path('test_embeds.npy')
    if train_embeds_path.exists() and test_embeds_path.exists():
        train_embeds = np.load(train_embeds_path)
        test_embeds = np.load(test_embeds_path)
        print(f"Loaded cached embeddings: train {train_embeds.shape}, test {test_embeds.shape}")
    else:
        # Fallback to high-throughput extractor defined in Utility Library (Cell 2)
        try:
            _ = extract_embeds_dl
        except NameError:
            raise RuntimeError('extract_embeds_dl is not defined. Execute the Utility Library cell (Cell 2) before re-running this cell.')
        model_name = 'vit_base_patch16_224'
        train_ids_all = train_meta['id'].tolist()
        test_ids_all  = test_meta['id'].tolist()
        print('Computing embeddings via extract_embeds_dl over cached tensors...')
        t0 = time.time()
        train_embeds = extract_embeds_dl(train_ids_all, split='train', model_name=model_name, batch_size=256, num_workers=0)
        test_embeds  = extract_embeds_dl(test_ids_all,  split='test',  model_name=model_name, batch_size=256, num_workers=0)
        np.save(train_embeds_path, train_embeds)
        np.save(test_embeds_path,  test_embeds)
        print(f"Embeddings extracted in {time.time()-t0:.1f}s; dim={train_embeds.shape[1]}")

    # MANDATORY HARDENING: L2-normalize embeddings to make IP == cosine similarity
    train_embeds = train_embeds.astype(np.float32, copy=False)
    test_embeds = test_embeds.astype(np.float32, copy=False)
    train_norms = np.linalg.norm(train_embeds, axis=1, keepdims=True) + 1e-12
    test_norms = np.linalg.norm(test_embeds, axis=1, keepdims=True) + 1e-12
    train_embeds = train_embeds / train_norms
    test_embeds = test_embeds / test_norms

    # Build FAISS index over normalized embeddings
    d = train_embeds.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(train_embeds.astype('float32'))

    # Train-Test nearest neighbors (for reporting/coverage, not grouping)
    K = min(100, train_embeds.shape[0])
    sims, idxs = index.search(test_embeds.astype('float32'), K)
    nn_hits = []
    for i in range(test_embeds.shape[0]):
        test_id = test_meta.at[i, 'id']
        for rank in range(K):
            tr_idx = int(idxs[i, rank])
            sim = float(sims[i, rank])
            nn_hits.append({'test_id': test_id, 'train_id': train_meta.at[tr_idx, 'id'], 'sim': sim, 'rank': rank})
    nn_hits_df = pd.DataFrame(nn_hits)
    nn_hits_df.to_csv('nn_hits.csv', index=False)

    # Intra-train nearest neighbors to form candidate pairs above threshold (will be filtered with safeguards)
    sims_tt, idxs_tt = index.search(train_embeds.astype('float32'), K)
    FAISS_TRAIN_SIM_THR = 0.985
    faiss_train_pairs_raw = []  # (i, j, sim)
    for i in range(train_embeds.shape[0]):
        for rank in range(1, K):
            j = int(idxs_tt[i, rank])
            sim = float(sims_tt[i, rank])
            if sim >= FAISS_TRAIN_SIM_THR:
                a, b = (i, j) if i < j else (j, i)
                faiss_train_pairs_raw.append((a, b, sim))
    faiss_train_pairs_raw = list({(a,b):sim for a,b,sim in faiss_train_pairs_raw}.items())
    faiss_train_pairs_raw = [(a, b, sim) for (a,b), sim in faiss_train_pairs_raw]
    print(f"FAISS intra-train candidate pairs (sim>={FAISS_TRAIN_SIM_THR}): {len(faiss_train_pairs_raw)}")

    # Safeguard filtering for FAISS unions: only same-breed, or corroborated by strong phash (<=2)
    train_ids = train_meta['id'].tolist()
    same_breed = lambda x, y: id_to_breed[train_ids[x]] == id_to_breed[train_ids[y]]
    phash_low_set = phash_corrob_low
    faiss_train_pairs = []
    STRICT_CROSS_BREED_SIM = 0.998  # much stricter if breeds differ and no corroboration
    for a, b, sim in faiss_train_pairs_raw:
        if same_breed(a, b) or ((min(a,b), max(a,b)) in phash_low_set):
            faiss_train_pairs.append((a, b, sim))
        elif sim >= STRICT_CROSS_BREED_SIM:
            faiss_train_pairs.append((a, b, sim))
    print(f"FAISS intra-train pairs after safeguards: {len(faiss_train_pairs)}")

    # 3) Union-Find consolidation of groups (md5/phash eq + phash ham + FAISS)
    # Rebuild exact md5 and identical phash maps from meta
    md5_to_ids = defaultdict(list)
    phash_to_ids = defaultdict(list)
    for _, r in train_meta.iterrows():
        md5_to_ids[r['md5']].append(r['id'])
        if pd.notna(r['phash']): phash_to_ids[r['phash']].append(r['id'])

    # Map train id to index
    train_id_to_idx = {id_: i for i, id_ in enumerate(train_meta['id'].tolist())}

    parent = list(range(len(train_meta)))
    rank = [0]*len(train_meta)
    methods_per_edge = defaultdict(set)

    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(x, y, method):
        rx, ry = find(x), find(y)
        if rx == ry:
            methods_per_edge[rx].add(method)
            return
        if rank[rx] < rank[ry]:
            parent[rx] = ry
            methods_per_edge[ry].update(methods_per_edge[rx]); methods_per_edge[ry].add(method)
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
            methods_per_edge[rx].update(methods_per_edge[ry]); methods_per_edge[rx].add(method)
        else:
            parent[ry] = rx
            rank[rx] += 1
            methods_per_edge[rx].update(methods_per_edge[ry]); methods_per_edge[rx].add(method)

    # md5 equality unions
    for md5v, ids in md5_to_ids.items():
        if len(ids) > 1:
            base = train_id_to_idx[ids[0]]
            for other in ids[1:]:
                union(base, train_id_to_idx[other], 'md5')

    # identical phash unions
    for ph, ids in phash_to_ids.items():
        if len(ids) > 1:
            base = train_id_to_idx[ids[0]]
            for other in ids[1:]:
                union(base, train_id_to_idx[other], 'phash_eq')

    # phash near hamming unions (high-recall from binary search)
    for ia, ib, ham in phash_train_pairs:
        union(ia, ib, 'phash_ham')

    # faiss intra-train unions (after safeguards)
    for ia, ib, sim in faiss_train_pairs:
        union(ia, ib, 'faiss')

    # Build groups (split mixed-label components into breed-pure subgroups)
    root_to_members = defaultdict(list)
    for i in range(len(train_meta)):
        r = find(i)
        root_to_members[r].append(i)

    groups = []
    id_to_group = {}
    mixed_groups = 0
    gid_counter = 0
    for root, members in root_to_members.items():
        # Partition by breed to ensure purity
        breed_buckets = defaultdict(list)
        for idx in members:
            _id = train_meta.at[idx, 'id']
            br = id_to_breed[_id]
            breed_buckets[br].append(idx)
        # detection methods observed for this UF component
        method_set_root = sorted(list(methods_per_edge[root])) if root in methods_per_edge else []
        for br, idx_list in breed_buckets.items():
            ids = [train_meta.at[i, 'id'] for i in idx_list]
            group_id = f"grp_{gid_counter}"
            gid_counter += 1
            groups.append({
                'group_id': group_id,
                'size': len(ids),
                'ids': ids,
                'detection_methods': method_set_root,
                'label_purity': True,
                'breeds': [br],
            })
            for _id in ids:
                id_to_group[_id] = group_id

    with open('dup_groups.json', 'w') as f:
        json.dump({'groups': groups, 'id_to_group': id_to_group}, f)
    print(f"Consolidated duplicate groups after purity split: {len(groups)} (all train samples assigned)")

    # 4) Produce complete group assignment artifact
    train_groups = pd.DataFrame({
        'id': train_meta['id'],
        'breed': train_meta['id'].map(id_to_breed),
        'group_id': train_meta['id'].map(id_to_group)
    })
    assert train_groups['group_id'].isna().sum() == 0, 'All images must have a group id'

    # Verify zero mixed-label groups
    grp_breeds = train_groups.groupby('group_id')['breed'].nunique()
    mixed_ct = int((grp_breeds > 1).sum())
    assert mixed_ct == 0, f'Mixed-label groups remain: {mixed_ct}'
    train_groups.to_csv('train_groups.csv', index=False)
    print('Saved train_groups.csv (purity enforced: 0 mixed groups)')

    # 5) Augment train_test_sim_hits with md5/phash_eq + phash_ham + FAISS
    hits_eq = []
    # md5 equality from exact maps
    for md5v, ids in md5_to_ids.items():
        test_match = test_meta[test_meta['md5'] == md5v]['id'].tolist()
        if not test_match:
            continue
        for t in test_match:
            for tr in ids:
                hits_eq.append({'test_id': t, 'train_id': tr, 'type': 'md5_eq', 'distance': 0.0})

    # identical phash
    for ph, ids in phash_to_ids.items():
        if pd.isna(ph):
            continue
        test_match = test_meta[test_meta['phash'] == ph]['id'].tolist()
        if not test_match:
            continue
        for t in test_match:
            for tr in ids:
                hits_eq.append({'test_id': t, 'train_id': tr, 'type': 'phash_eq', 'distance': 0.0})

    # add phash_ham (high-recall)
    hits_ph_ham = phash_tt_hits

    # FAISS test->train above thresholds
    FAISS_TEST_SIM_THR_HARD = 0.995
    FAISS_TEST_SIM_THR_SOFT = 0.985
    hits_faiss = []
    if 'nn_hits_df' in locals() and len(nn_hits_df) > 0:
        for row in nn_hits_df.itertuples(index=False):
            sim = row.sim
            if sim >= FAISS_TEST_SIM_THR_SOFT:
                hits_faiss.append({
                    'test_id': row.test_id,
                    'train_id': row.train_id,
                    'type': 'faiss',
                    'distance': 1.0 - float(sim),
                })

    aug_hits = pd.DataFrame(hits_eq + hits_ph_ham + hits_faiss)
    if len(aug_hits) == 0:
        aug_hits = pd.DataFrame(columns=['test_id','train_id','type','distance'])
    aug_hits['train_breed'] = aug_hits['train_id'].map(id_to_breed)
    aug_hits.to_csv('train_test_sim_hits.csv', index=False)
    print(f"Augmented train_test_sim_hits.csv with {len(aug_hits)} records")

    # Coverage metrics
    if 'nn_hits_df' in locals() and len(nn_hits_df) > 0:
        max_sim = nn_hits_df.groupby('test_id')['sim'].max()
        cov_hard = (max_sim >= FAISS_TEST_SIM_THR_HARD).mean()
        cov_soft = (max_sim >= FAISS_TEST_SIM_THR_SOFT).mean()
        print(f"FAISS coverage: hard(>= {FAISS_TEST_SIM_THR_HARD})={cov_hard*100:.1f}% | soft(>= {FAISS_TEST_SIM_THR_SOFT})={cov_soft*100:.1f}% of test")

    print('Advanced forensics remediations complete: high-recall phash, K increased, safeguarded FAISS unions, purity enforced (0 mixed groups). Ready for audit.')


Forensics artifacts detected (train_groups.csv, train_test_sim_hits.csv, nn_hits.csv). Skipping recomputation.


In [4]:
# EDA summary + StratifiedGroupKFold fold assignment using purity-enforced groups
import os, json, random
from pathlib import Path
import numpy as np
import pandas as pd
from collections import Counter

SEED = 20250810
random.seed(SEED); np.random.seed(SEED)

BASE = Path('.')
labels_df = pd.read_csv('labels.csv')
train_groups = pd.read_csv('train_groups.csv')  # id, breed, group_id
sample_df = pd.read_csv('sample_submission.csv')
classes = [c for c in sample_df.columns if c != 'id']

# Merge labels with groups
df = labels_df.merge(train_groups[['id','group_id']], on='id', how='left')
assert df['group_id'].isna().sum() == 0, 'Missing group_id assignments'

# Basic EDA summaries
num_classes = df['breed'].nunique()
num_samples = len(df)
num_groups = df['group_id'].nunique()
class_counts = df['breed'].value_counts().to_dict()
group_sizes = df.groupby('group_id').size()
eda = {
    'num_classes': int(num_classes),
    'num_samples': int(num_samples),
    'num_groups': int(num_groups),
    'group_size_summary': group_sizes.describe().to_dict(),
    'class_count_min': int(min(class_counts.values())),
    'class_count_max': int(max(class_counts.values())),
}
with open('eda_summary.json','w') as f:
    json.dump(eda, f)
print('EDA:', eda)

# Build StratifiedGroupKFold
from sklearn.model_selection import StratifiedGroupKFold
skf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)

# Prepare arrays
X = np.zeros((len(df), 1))  # dummy
y = df['breed'].values
groups = df['group_id'].values

folds = np.full(len(df), -1, dtype=int)
for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y, groups)):
    folds[va_idx] = fold

assert (folds >= 0).all(), 'All samples must be assigned a fold'
df['fold'] = folds

# Validate: no group crosses folds
grp_to_folds = df.groupby('group_id')['fold'].nunique()
crossing = int((grp_to_folds > 1).sum())
assert crossing == 0, f'Groups crossing folds detected: {crossing}'

# Inspect stratification quality
per_fold_counts = df.groupby(['fold','breed']).size().unstack(fill_value=0)
fold_sizes = df['fold'].value_counts().sort_index().to_dict()
print('Fold sizes:', fold_sizes)

# Correct stratification check: compare each fold's class distribution to the overall dataset distribution
overall_dist = df['breed'].value_counts(normalize=True)  # index: breed
fold_dists = df.groupby('fold')['breed'].value_counts(normalize=True).unstack(fill_value=0)  # index: fold, columns: breed
# Align overall distribution to columns of fold_dists
overall_vec = overall_dist.reindex(fold_dists.columns).fillna(0)
# Broadcast subtract row-wise (each fold row minus overall vector)
imbalance_correct = (fold_dists.sub(overall_vec, axis=1).abs()).mean().mean()
print(f'Corrected Mean Absolute Deviation from Overall Class Distribution: {imbalance_correct:.6f}')

# Persist fold assignments
df[['id','breed','group_id','fold']].to_csv('fold_assignments.csv', index=False)
print('Saved fold_assignments.csv with 5 folds, group-based stratification and zero group leakage.')

# Acknowledge NN coverage is tiny; treat retrieval blending as polish, not core gains
print('Note: FAISS test coverage was very low in forensics (\u22480.3% hard, 0.8% soft). NN blending will be used cautiously as minor polish.')

# Lightweight notebook backup before modeling stage
try:
    import shutil
    ts = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
    nb_src = 'agent_notebook.ipynb'
    if Path(nb_src).exists():
        shutil.copyfile(nb_src, f'nb_backup_before_modeling_{ts}.ipynb')
        print('Notebook backup created:', f'nb_backup_before_modeling_{ts}.ipynb')
except Exception as e:
    print('Notebook backup skipped:', e)

print('EDA + Fold assignment complete. Ready for modeling baseline.')


EDA: {'num_classes': 120, 'num_samples': 9199, 'num_groups': 9152, 'group_size_summary': {'count': 9152.0, 'mean': 1.0051354895104896, 'std': 0.08281375105293992, 'min': 1.0, '25%': 1.0, '50%': 1.0, '75%': 1.0, 'max': 4.0}, 'class_count_min': 58, 'class_count_max': 118}


Fold sizes: {0: 1837, 1: 1838, 2: 1836, 3: 1847, 4: 1841}
Corrected Mean Absolute Deviation from Overall Class Distribution: 0.001553
Saved fold_assignments.csv with 5 folds, group-based stratification and zero group leakage.
Note: FAISS test coverage was very low in forensics (≈0.3% hard, 0.8% soft). NN blending will be used cautiously as minor polish.
Notebook backup created: nb_backup_before_modeling_20250812_173254.ipynb
EDA + Fold assignment complete. Ready for modeling baseline.


In [5]:
# Fast Baseline (Corrected): Linear classifier on cached embeddings + 5-fold SGKF + temperature scaling
# Critical fixes:
# - Ensure embeddings, labels, and folds are perfectly aligned by id (fatal bug previously)
# - Overwrite prior invalid artifacts
# - SINGLE-SOURCE-OF-TRUTH ENFORCEMENT: Do NOT write submission.csv here; write submission_baseline.csv only (diagnostic).

import os, json, math, random, time
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

SEED = 20250810
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load metadata and artifacts
labels_df = pd.read_csv('labels.csv')
sample_df = pd.read_csv('sample_submission.csv')
classes = [c for c in sample_df.columns if c != 'id']
breed_to_idx = {b:i for i,b in enumerate(classes)}
folds_df = pd.read_csv('fold_assignments.csv')  # id, breed, group_id, fold
train_meta = pd.read_csv('train_image_meta.csv')
test_meta  = pd.read_csv('test_image_meta.csv')

# Load cached embeddings (in the order used during extraction: train_meta/test_meta order)
train_embeds_raw = np.load('train_embeds.npy')  # shape (N, D)
test_embeds_raw  = np.load('test_embeds.npy')   # shape (M, D)

# Sanity checks: same sets of ids
train_ids_extract = train_meta['id'].tolist()
train_ids_labels  = labels_df['id'].tolist()
assert set(train_ids_extract) == set(train_ids_labels), 'Mismatch between ids in train_image_meta.csv and labels.csv'
assert train_embeds_raw.shape[0] == len(train_ids_extract), 'train_embeds rows must match train_image_meta rows'
assert test_embeds_raw.shape[0] == len(test_meta), 'test_embeds rows must match test_image_meta rows'

# ALIGNMENT: reorder train embeddings to match labels_df id order; align folds to the same order
id_to_pos = {id_: i for i, id_ in enumerate(train_ids_extract)}
reindex = np.array([id_to_pos[id_] for id_ in train_ids_labels], dtype=np.int64)
train_embeds = train_embeds_raw[reindex]
y = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
folds = folds_df.set_index('id').loc[train_ids_labels, 'fold'].values.astype(int)

# Verify alignment consistency
assert train_embeds.shape[0] == len(y) == len(folds) == len(train_ids_labels)
print('Alignment OK: embeddings, labels, and folds share identical id order.')

# For test, keep the same order as test_meta; submission will use test_meta['id']
test_embeds = test_embeds_raw.astype(np.float32)

n_classes = len(classes)

class BreedEmbedDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X.astype(np.float32)
        self.y = None if y is None else y.astype(np.int64)
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        if self.y is None:
            return torch.from_numpy(self.X[idx])
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx])

def train_linear_head(X_tr, y_tr, X_va, y_va, num_classes, epochs=60, bs=2048, lr=2e-3, wd=0.02, patience=10):
    model = nn.Linear(X_tr.shape[1], num_classes).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)
    crit = nn.CrossEntropyLoss()
    dl_tr = DataLoader(BreedEmbedDataset(X_tr, y_tr), batch_size=bs, shuffle=True, num_workers=2, pin_memory=True)
    dl_va = DataLoader(BreedEmbedDataset(X_va, y_va), batch_size=bs*2, shuffle=False, num_workers=2, pin_memory=True)
    best_state, best_loss, no_improve = None, 1e9, 0
    for ep in range(epochs):
        model.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = crit(logits, yb)
            loss.backward()
            opt.step()
        model.eval()
        va_loss = 0.0; n = 0
        with torch.no_grad():
            for xb, yb in dl_va:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                loss = crit(logits, yb)
                va_loss += loss.item() * xb.size(0)
                n += xb.size(0)
        va_loss /= max(1, n)
        sched.step()
        if va_loss + 1e-7 < best_loss:
            best_loss = va_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# 5-fold SGKF using precomputed folds
n_folds = int(folds.max() + 1)
oof_logits = np.zeros((train_embeds.shape[0], n_classes), dtype=np.float32)
test_logits_f = np.zeros((n_folds, test_embeds.shape[0], n_classes), dtype=np.float32)

for f in range(n_folds):
    tr_idx = np.where(folds != f)[0]
    va_idx = np.where(folds == f)[0]
    X_tr, y_tr = train_embeds[tr_idx], y[tr_idx]
    X_va, y_va = train_embeds[va_idx], y[va_idx]
    model = train_linear_head(X_tr, y_tr, X_va, y_va, n_classes)
    with torch.no_grad():
        va_logits = model(torch.from_numpy(X_va).to(device)).float().cpu().numpy()
        oof_logits[va_idx] = va_logits
        te_logits = model(torch.from_numpy(test_embeds).to(device)).float().cpu().numpy()
        test_logits_f[f] = te_logits
    print(f"Fold {f}: val logits collected: {va_logits.shape}")

test_logits = test_logits_f.mean(axis=0)
np.save('oof_logits.npy', oof_logits)
np.save('test_logits.npy', test_logits)

# Fit temperature on OOF and apply (using canonical utility from Cell 1)
T = fit_temperature(oof_logits, y)
with open('temperatures.json', 'w') as f:
    json.dump({'global_T': T}, f)
print(f'Temperature fitted on OOF: T={T:.4f}')

oof_probs = softmax_np(oof_logits / T)
np.save('oof_probs.npy', oof_probs)

test_probs = softmax_np(test_logits / T)
# Clamp for numerical safety
eps = 1e-8
test_probs = np.clip(test_probs, eps, 1.0)
test_probs = test_probs / test_probs.sum(axis=1, keepdims=True)

# Build diagnostic baseline submission aligned to test_meta ids
sub = pd.DataFrame(test_probs, columns=classes)
sub.insert(0, 'id', test_meta['id'])
sub.to_csv('submission_baseline.csv', index=False)
print('submission_baseline.csv written (diagnostic only):', sub.shape)

# Quick OOF log loss report (uncalibrated vs calibrated) using canonical utility
oof_probs_uncal = softmax_np(oof_logits)
ll_uncal = log_loss_np(y, oof_probs_uncal)
ll_cal = log_loss_np(y, oof_probs)
print(f'OOF log loss (uncalibrated): {ll_uncal:.6f} | (calibrated): {ll_cal:.6f}')

print('Corrected baseline complete (aligned embeddings). Single-source-of-truth enforced: final submission.csv is produced only by Cell 8.')


Alignment OK: embeddings, labels, and folds share identical id order.


/app/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Fold 0: val logits collected: (1837, 120)


Fold 1: val logits collected: (1838, 120)


Fold 2: val logits collected: (1836, 120)


Fold 3: val logits collected: (1847, 120)


Fold 4: val logits collected: (1841, 120)


Temperature fitted on OOF: T=0.1816


submission_baseline.csv written (diagnostic only): (1023, 121)
OOF log loss (uncalibrated): 3.007870 | (calibrated): 0.704459
Corrected baseline complete (aligned embeddings). Single-source-of-truth enforced: final submission.csv is produced only by Cell 8.


In [6]:
# Diagnostic Base Submission Only — Single-source-of-truth enforced
# This cell writes a simple diagnostic submission directly from the strongest base logits (full-image preferred, else baseline).
# It performs temperature application only (fit if missing) and writes submission_base_diag.csv.
# All kNN blending, tuning, and final submission.csv creation are exclusively handled by Cell 8.

import os, json
from pathlib import Path
import numpy as np
import pandas as pd
import torch

SEED = 20250810
torch.manual_seed(SEED)

sample_df = pd.read_csv('sample_submission.csv')
classes = [c for c in sample_df.columns if c != 'id']
labels_df = pd.read_csv('labels.csv')
breed_to_idx = {b:i for i,b in enumerate(classes)}
y = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
test_meta = pd.read_csv('test_image_meta.csv')
test_ids = test_meta['id'].tolist()

# Detect strongest available base artifacts (full-image preferred)
full_oof = Path('oof_logits_fullimg.npy')
full_te  = Path('test_logits_fullimg.npy')
full_T   = Path('temperatures_fullimg.json')
base_oof = Path('oof_logits.npy')
base_te  = Path('test_logits.npy')
base_T   = Path('temperatures.json')

use_full = full_oof.exists() and full_te.exists()
base_source = 'fullimg'
if use_full:
    oof_logits = np.load(full_oof)
    te_logits  = np.load(full_te)
    # Guard against placeholders (zeros from CPU short-circuit)
    if np.allclose(oof_logits, 0.0) or np.allclose(te_logits, 0.0):
        print('Diagnostic: detected placeholder full-image logits. Falling back to baseline.')
        use_full = False

if not use_full:
    if not (base_oof.exists() and base_te.exists()):
        raise FileNotFoundError('Baseline artifacts missing. Run the baseline cell to generate oof_logits.npy and test_logits.npy.')
    base_source = 'baseline'
    oof_logits = np.load(base_oof)
    te_logits  = np.load(base_te)

# Ensure temperature exists; fit if missing
def _load_T(path: Path):
    if path.exists():
        try:
            return float(json.load(open(path))['global_T'])
        except Exception:
            return None
    return None
T_file = full_T if use_full else base_T
T = _load_T(T_file)
if T is None or not np.isfinite(T):
    print('Diagnostic: fitting temperature for base OOF logits...')
    T = fit_temperature(oof_logits, y)
    with open(T_file, 'w') as f:
        json.dump({'global_T': float(T)}, f)

# Build diagnostic base submission with calibrated probabilities
probs_test = softmax_np(te_logits / max(1e-3, T))
probs_test = row_normalize(probs_test)
sub = pd.DataFrame(probs_test, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission_base_diag.csv', index=False)
print(f'Diagnostic base submission written: submission_base_diag.csv (source={base_source}, T={T:.4f}).')

print('Note: kNN blending and final submission.csv are produced only by Cell 8 (Autonomous kNN).')


Diagnostic: detected placeholder full-image logits. Falling back to baseline.


Diagnostic base submission written: submission_base_diag.csv (source=baseline, T=0.1816).
Note: kNN blending and final submission.csv are produced only by Cell 8 (Autonomous kNN).


In [7]:
# Full-Image Training Pipeline (GPU-first) — Parametric backbone @384 with SGKF folds, AMP, Mixup/CutMix, EMA, TTA
# Audit fixes: ema_model.eval() for val/infer, deterministic settings, seeded workers (including torch), aspect-ratio eval transform,
# DataLoader perf flags, parametric model_name & TTA, robust artifacting (checkpoints + manifest). Now saves canonical artifact names.

import time, json, random, shutil, math
from pathlib import Path
from typing import Optional, List, Callable, Dict, Any, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image, ImageOps
import timm
from timm.loss import SoftTargetCrossEntropy
from timm.utils import ModelEmaV2
from timm.data import Mixup

SEED = 20250810
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('CUDA available:', torch.cuda.is_available(), '| device:', device)
if device != 'cuda':
    print('WARNING: CUDA not available; training will be extremely slow. If this persists, please re-attach GPU.')

# Data & labels
labels_df = pd.read_csv('labels.csv')
folds_df = pd.read_csv('fold_assignments.csv')  # id, breed, group_id, fold
train_ids = labels_df['id'].tolist()
test_meta = pd.read_csv('test_image_meta.csv')
test_ids = test_meta['id'].tolist()
sample_df = pd.read_csv('sample_submission.csv')
classes = [c for c in sample_df.columns if c != 'id']
breed_to_idx = {b:i for i,b in enumerate(classes)}
y_all = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
folds = folds_df.set_index('id').loc[labels_df['id'], 'fold'].values.astype(int)
n_classes = len(classes)
n_folds = int(folds.max() + 1)

# CPU short-circuit path when CUDA is unavailable: create deterministic placeholder artifacts but DO NOT terminate the kernel
if device != 'cuda':
    print('CPU short-circuit: emitting placeholder full-image artifacts to keep notebook reproducible.')
    # Placeholder logits (canonical names)
    oof_placeholder = np.zeros((len(train_ids), n_classes), dtype=np.float32)
    test_placeholder = np.zeros((len(test_ids), n_classes), dtype=np.float32)
    np.save('oof_logits_fullimg.npy', oof_placeholder)
    np.save('test_logits_fullimg.npy', test_placeholder)
    # Placeholder embeddings (1-dim to avoid guessing feature size)
    np.save('oof_embeds_fullimg.npy', np.zeros((len(train_ids), 1), dtype=np.float32))
    np.save('test_embeds_fullimg.npy', np.zeros((len(test_ids), 1), dtype=np.float32))
    # Temperature manifest (canonical)
    with open('temperatures_fullimg.json','w') as f:
        json.dump({'global_T': 1.0, 'note': 'CPU-shortcircuit; no training performed'}, f)
    # Run manifest
    manifest = {
        'seed': SEED,
        'device': device,
        'classes': n_classes,
        'folds': n_folds,
        'config': {'model_name': 'convnext_base', 'img_size': 384, 'epochs': 0, 'bs': 0, 'lr': 0.0, 'wd': 0.0},
        'fold_val_losses': [],
        'note': 'Skipped training due to CUDA unavailable.'
    }
    with open('run_manifest_convnext_base_sz384_seed20250810.json', 'w') as f:
        json.dump(manifest, f)
    # submission_fullimg.csv: copy calibrated baseline if available; else uniform probs
    if Path('submission.csv').exists():
        shutil.copyfile('submission.csv', 'submission_fullimg.csv')
        print('Copied existing submission.csv -> submission_fullimg.csv (baseline passthrough).')
    else:
        P = np.full((len(test_ids), n_classes), 1.0 / max(1, n_classes), dtype=np.float32)
        sub_stub = pd.DataFrame(P, columns=classes)
        sub_stub.insert(0, 'id', test_ids)
        sub_stub.to_csv('submission_fullimg.csv', index=False)
        print('Wrote uniform-probability submission_fullimg.csv stub (no baseline found).')
    print('CPU short-circuit complete. Proceeding without training (no SystemExit).')

def _worker_init_fn(worker_id: int):
    base_seed = SEED + worker_id
    np.random.seed(base_seed)
    random.seed(base_seed)
    torch.manual_seed(base_seed)  # Reproducibility: seed torch in workers

class DogDataset(Dataset):
    def __init__(self, ids, split: str, labels: Optional[np.ndarray] = None, img_size: int = 384, aug: bool = False):
        self.ids = ids
        self.split = split
        self.labels = labels
        self.root = Path('train' if split=='train' else 'test')
        if aug:
            self.tf = T.Compose([
                T.Resize(int(img_size*1.1), interpolation=T.InterpolationMode.BICUBIC),
                T.RandomResizedCrop(img_size, scale=(0.8, 1.0), interpolation=T.InterpolationMode.BICUBIC),
                T.RandomHorizontalFlip(p=0.5),
                T.RandAugment(num_ops=2, magnitude=9),
                T.ToTensor(),
                T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
                T.RandomErasing(p=0.25, value='random')
            ])
        else:
            # Aspect-ratio preserving eval pipeline
            self.tf = T.Compose([
                T.Resize(int(img_size*1.1), interpolation=T.InterpolationMode.BICUBIC),
                T.CenterCrop(img_size),
                T.ToTensor(),
                T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
            ])
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, i):
        id_ = self.ids[i]
        img = Image.open(self.root / f"{id_}.jpg")
        img = ImageOps.exif_transpose(img)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        x = self.tf(img)
        img.close()
        if self.labels is None:
            return x, id_
        return x, self.labels[i]

def get_mixup_fn(num_classes: int) -> Mixup:
    return Mixup(mixup_alpha=0.2, cutmix_alpha=0.1, prob=1.0, switch_prob=0.5, mode='batch', label_smoothing=0.05, num_classes=num_classes)

def create_model(model_name: str, num_classes: int, drop_path: float = 0.2):
    return timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_path_rate=drop_path)

def _extract_features_backbone(ema_model: torch.nn.Module, xb: torch.Tensor) -> torch.Tensor:
    # Extract penultimate features using timm's forward_features then global pool
    if hasattr(ema_model, 'forward_features'):
        feats = ema_model.forward_features(xb)
    else:
        # Fallback: use model minus classifier if available; here we rely on forward_features for timm models
        feats = ema_model(xb)
    # Global pooling if spatial
    if feats.ndim == 4:
        feats = feats.mean(dim=(2,3))
    return feats

def _build_warmup_cosine_scheduler(optimizer, epochs, iters_per_epoch, warmup_epochs=2):
    total_steps = max(1, epochs * iters_per_epoch)
    warm_steps = int(max(0, warmup_epochs) * iters_per_epoch)
    def lr_lambda(step: int):
        if warm_steps > 0 and step < warm_steps:
            return float(step + 1) / float(max(1, warm_steps))
        # cosine phase
        prog = 0.0 if total_steps == warm_steps else float(step - warm_steps) / float(max(1, total_steps - warm_steps))
        return 0.5 * (1.0 + math.cos(math.pi * min(1.0, max(0.0, prog))))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

def _make_tta_list(scales: List[int]) -> List[Callable[[torch.Tensor], torch.Tensor]]:
    # Returns TTAs: for each scale, identity and hflip at that resolution
    tta_fns: List[Callable[[torch.Tensor], torch.Tensor]] = []
    for sz in scales:
        def resize_fn(x, s=sz):
            if x.shape[-1] == s and x.shape[-2] == s:
                return x
            return F.interpolate(x, size=(s, s), mode='bilinear', align_corners=False)
        tta_fns.append(lambda x, rf=resize_fn: rf(x))
        tta_fns.append(lambda x, rf=resize_fn: torch.flip(rf(x), dims=[3]))  # hflip after resize
    return tta_fns

def train_one_fold(fold:int, model_name: str, img_size=384, epochs=8, bs=32, lr=1e-3, wd=2e-2, warmup_epochs: int = 2,
                   tta_fns: Optional[List[Callable[[torch.Tensor], torch.Tensor]]] = None,
                   num_workers: int = 8) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float, np.ndarray, np.ndarray]:
        
    tr_idx = np.where(folds != fold)[0]
    va_idx = np.where(folds == fold)[0]
    ids_tr = [train_ids[i] for i in tr_idx]
    ids_va = [train_ids[i] for i in va_idx]
    y_tr = y_all[tr_idx]
    y_va = y_all[va_idx]

    ds_tr = DogDataset(ids_tr, 'train', y_tr, img_size=img_size, aug=True)
    ds_va = DogDataset(ids_va, 'train', y_va, img_size=img_size, aug=False)
    dl_tr = DataLoader(ds_tr, batch_size=bs, shuffle=True, num_workers=num_workers, pin_memory=True,
                       worker_init_fn=_worker_init_fn, persistent_workers=True, prefetch_factor=2)
    dl_va = DataLoader(ds_va, batch_size=bs*2, shuffle=False, num_workers=num_workers, pin_memory=True,
                       worker_init_fn=_worker_init_fn, persistent_workers=True, prefetch_factor=2)

    model = create_model(model_name, n_classes, drop_path=0.2).to(device)
    ema = ModelEmaV2(model, decay=0.9998, device=device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    iters_per_epoch = max(1, len(dl_tr))
    sched = _build_warmup_cosine_scheduler(opt, epochs=epochs, iters_per_epoch=iters_per_epoch, warmup_epochs=warmup_epochs)
    mixup_fn = get_mixup_fn(n_classes)
    criterion = SoftTargetCrossEntropy()
    scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))

    best_loss = 1e9
    best_state = None

    global_step = 0
    for ep in range(epochs):
        model.train()
        t0 = time.time()
        for xb, yb in dl_tr:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            xb, yb_mix = mixup_fn(xb, yb)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(device=='cuda')):
                logits = model(xb)
                loss = criterion(logits, yb_mix)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
            ema.update(model)
            # step warmup+cosine every iteration
            sched.step()
            global_step += 1

        # validation
        model.eval()
        ema_model = ema.module
        ema_model.eval()  # CRITICAL: ensure eval mode for EMA during validation
        val_loss = 0.0; n = 0
        with torch.no_grad():
            for xb, yb in dl_va:
                xb = xb.to(device, non_blocking=True)
                yb = yb.to(device, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(device=='cuda')):
                    logits = ema_model(xb)
                    loss = nn.CrossEntropyLoss()(logits, yb)
                val_loss += loss.item() * xb.size(0)
                n += xb.size(0)
        val_loss /= max(1, n)
        dt = time.time() - t0
        print(f"Fold {fold} | Epoch {ep+1}/{epochs} | val_loss={val_loss:.5f} | {dt:.1f}s")
        if val_loss + 1e-6 < best_loss:
            best_loss = val_loss
            best_state = {k: v.detach().cpu() for k, v in ema_model.state_dict().items()}

    # load best EMA state
    if best_state is not None:
        ema_model.load_state_dict(best_state, strict=False)

    # Save best EMA checkpoint for this fold
    ckpt_name = f"ckpt_{model_name}_sz{img_size}_seed{SEED}_fold{fold}.pt"
    torch.save({'model_name': model_name, 'state_dict': {k: v.cpu() for k, v in ema_model.state_dict().items()},
                'img_size': img_size, 'seed': SEED, 'fold': fold, 'val_loss': float(best_loss)}, ckpt_name)
    print('Saved checkpoint:', ckpt_name)

    # collect OOF logits and embeddings for this fold
    oof_logits_fold = np.zeros((len(va_idx), n_classes), dtype=np.float32)
    oof_embeds_fold: Optional[np.ndarray] = None
    ema_model.eval()  # ensure eval for inference
    with torch.no_grad():
        ptr = 0
        for xb, yb in dl_va:
            xb = xb.to(device, non_blocking=True)
            logits = ema_model(xb)
            feats = _extract_features_backbone(ema_model, xb)
            logits_np = logits.float().cpu().numpy()
            feats_np = feats.float().cpu().numpy()
            if oof_embeds_fold is None:
                oof_embeds_fold = np.zeros((len(va_idx), feats_np.shape[1]), dtype=np.float32)
            oof_logits_fold[ptr:ptr+logits_np.shape[0]] = logits_np
            oof_embeds_fold[ptr:ptr+feats_np.shape[0]] = feats_np
            ptr += logits_np.shape[0]

    # test-time logits and embeddings with stronger multi-scale TTA (384/448 with hflip)
    if tta_fns is None or len(tta_fns) == 0:
        tta_fns = _make_tta_list([img_size, 448])
    ds_te = DogDataset(test_ids, 'test', labels=None, img_size=img_size, aug=False)
    dl_te = DataLoader(ds_te, batch_size=bs*2, shuffle=False, num_workers=num_workers, pin_memory=True,
                       worker_init_fn=_worker_init_fn, persistent_workers=True, prefetch_factor=2)
    test_logits_fold = np.zeros((len(test_ids), n_classes), dtype=np.float32)
    test_embeds_fold: Optional[np.ndarray] = None
    ema_model.eval()  # CRITICAL: eval mode during inference
    with torch.no_grad():
        ofs = 0
        for xb, _ in dl_te:
            xb = xb.to(device, non_blocking=True)
            logits_acc = None
            feats_acc = None
            for tta in tta_fns:
                x_aug = tta(xb)
                logits_aug = ema_model(x_aug)
                feats_aug = _extract_features_backbone(ema_model, x_aug)
                logits_acc = logits_aug if logits_acc is None else (logits_acc + logits_aug)
                feats_acc = feats_aug if feats_acc is None else (feats_acc + feats_aug)
            logits = (logits_acc / len(tta_fns)).float().cpu().numpy()
            feats = (feats_acc / len(tta_fns)).float().cpu().numpy()
            n_b = logits.shape[0]
            if test_embeds_fold is None:
                test_embeds_fold = np.zeros((len(test_ids), feats.shape[1]), dtype=np.float32)
            test_logits_fold[ofs:ofs+n_b] = logits
            test_embeds_fold[ofs:ofs+n_b] = feats
            ofs += n_b

    return va_idx, oof_logits_fold, test_logits_fold, float(best_loss), oof_embeds_fold, test_embeds_fold

# Guard the training orchestration to run ONLY on GPU
if device == 'cuda':
    # Orchestrate 5-fold run (GPU validation run). Parameterize backbone and TTA.
    config: Dict[str, Any] = {
        'model_name': 'convnext_base',
        'img_size': 384,
        'epochs': 2,  # SHORT GPU VALIDATION RUN per consolidated approval (was 20 for full baseline)
        'bs': 32 if device=='cuda' else 8,
        'lr': 1e-3,
        'wd': 2e-2,
        'num_workers': 8,
        'warmup_epochs': 2,
    }
    # Stronger TTA: multi-scale 384/448 with hflip variants (train-time aug remains at base img_size)
    tta_list = _make_tta_list([config['img_size'], 448])

    oof_logits = np.zeros((len(train_ids), n_classes), dtype=np.float32)
    test_logits_stack: Optional[np.ndarray] = None
    oof_embeds: Optional[np.ndarray] = None
    test_embeds_stack: list = []  # accumulate per-fold test embeddings, will mean at end
    fold_val_losses = []

    for f in range(n_folds):
        va_idx, oof_f, te_f, vloss, oof_e_f, te_e_f = train_one_fold(
            f,
            model_name=config['model_name'],
            img_size=config['img_size'],
            epochs=config['epochs'],
            bs=config['bs'],
            lr=config['lr'],
            wd=config['wd'],
            warmup_epochs=config['warmup_epochs'],
            tta_fns=tta_list,
            num_workers=config['num_workers']
        )
        # Initialize containers with known feature dim after first fold
        if test_logits_stack is None:
            test_logits_stack = np.zeros((n_folds, len(test_ids), n_classes), dtype=np.float32)
        if oof_embeds is None:
            oof_embeds = np.zeros((len(train_ids), oof_e_f.shape[1]), dtype=np.float32)
        oof_logits[va_idx] = oof_f
        oof_embeds[va_idx] = oof_e_f
        test_logits_stack[f] = te_f
        test_embeds_stack.append(te_e_f)
        fold_val_losses.append(float(vloss))
        # Save intermediate artifacts to be robust to interruptions (canonical names)
        np.save('oof_logits_fullimg.npy', oof_logits)
        np.save('test_logits_fullimg.npy', test_logits_stack.mean(axis=0))
        # Embeddings: mean test across folds, OOF assigned per fold
        np.save('oof_embeds_fullimg.npy', oof_embeds)
        np.save('test_embeds_fullimg.npy', np.mean(np.stack(test_embeds_stack, axis=0), axis=0))
        with open('throughput_benchmark.json','w') as f:
            json.dump({'fold_val_losses': fold_val_losses, **config}, f)
        print(f'Fold {f} complete. Best val loss: {vloss:.5f}')

    # Run manifest with hyperparameters and fold results
    manifest = {
        'seed': SEED,
        'device': device,
        'classes': n_classes,
        'folds': n_folds,
        'config': config,
        'fold_val_losses': fold_val_losses,
    }
    with open(f"run_manifest_{config['model_name']}_sz{config['img_size']}_seed{SEED}.json", 'w') as f:
        json.dump(manifest, f)

    # Temperature scaling on OOF logits (utility from Cell 1) — canonical filename
    T_full = fit_temperature(oof_logits, y_all)
    with open('temperatures_fullimg.json','w') as f:
        json.dump({'global_T': float(T_full), **config}, f)
    print(f'Full-image OOF temperature: T={T_full:.4f}')

    # Build calibrated submission from full-image model (canonical)
    test_logits = test_logits_stack.mean(axis=0)
    probs_test = softmax_np(test_logits / max(1e-3, T_full))
    probs_test = row_normalize(probs_test)
    sub_full = pd.DataFrame(probs_test, columns=classes)
    sub_full.insert(0, 'id', test_ids)
    sub_full.to_csv('submission_fullimg.csv', index=False)
    print('submission_fullimg.csv written:', sub_full.shape)

    print('Full-image short GPU validation run complete (epochs=2). Proceed to ensemble refactor after validating end-to-end execution.')


CUDA available: False | device: cpu
CPU short-circuit: emitting placeholder full-image artifacts to keep notebook reproducible.
Copied existing submission.csv -> submission_fullimg.csv (baseline passthrough).
CPU short-circuit complete. Proceeding without training (no SystemExit).


In [8]:
# FINAL AUTONOMOUS SUBMISSION: Post-training kNN hyperparameter search + calibrated blend (+ optional LR stacker)
# Purpose:
# - Single-pass autonomous pipeline: grid search (K, tau, lambda) using strongest embeddings (Multi > ConvNeXt > ViT)
#   and strongest base logits (full-image preferred), fit temperature on blended OOF, and write final submission.csv.
# - Adds a lightweight multinomial Logistic Regression stacker on OOF features [log-softmax(base), log(P_knn)]
#   with temperature scaling; selects the best of (blended vs stacked) by OOF log loss.
# - Output: submission.csv, knn_blend_config.json, temperatures_knn_blend.json

import json, os, time
from pathlib import Path
import numpy as np
import pandas as pd
import torch

# Mandatory Remediation B: resilient faiss import/install guard for clean environments
try:
    import faiss  # type: ignore
except Exception:
    import sys, subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-input', 'faiss-cpu'], check=True)
    import faiss  # type: ignore

SEED = 20250810
torch.manual_seed(SEED)

# CRITICAL GUARD: disable stacker to avoid meta-leak (audit directive)
DISABLE_STACKER = True
# Provenance-hardened: allow multi-encoder ONLY with manifest written by Cell 11 in current run
DISABLE_MULTI = False
# Methodology guards (per audit): keep test-time path identical to validated OOF path
ENABLE_TEST_LOCAL_RECAL = False  # if True, applies per-sample temperature by sim gap; must also be applied to OOF path (not implemented here)
ENABLE_HARD_OVERRIDES   = False  # if True, applies top-k consensus overrides; must be validated OOF-symmetric if enabled

# Load label/order metadata
labels_df = pd.read_csv('labels.csv')
sample_df = pd.read_csv('sample_submission.csv')
classes = [c for c in sample_df.columns if c != 'id']
breed_to_idx = {b:i for i,b in enumerate(classes)}
y = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
folds = pd.read_csv('fold_assignments.csv').set_index('id').loc[labels_df['id'], 'fold'].values.astype(int)
n_folds = int(folds.max() + 1)
test_meta = pd.read_csv('test_image_meta.csv')
test_ids = test_meta['id'].tolist()

# 1) Detect and load strongest available base logits (full-image preferred) with QUALITY GATE
full_oof_p = Path('oof_logits_fullimg.npy')
full_te_p  = Path('test_logits_fullimg.npy')
full_T_p   = Path('temperatures_fullimg.json')
base_oof_p = Path('oof_logits.npy')
base_te_p  = Path('test_logits.npy')
base_T_p   = Path('temperatures.json')

def _load_T(path: Path):
    if path.exists():
        try:
            return float(json.load(open(path))['global_T'])
        except Exception:
            return None
    return None

def _calibrated_loss(logits: np.ndarray, labels: np.ndarray, T: float) -> float:
    P = row_normalize(softmax_np(logits / max(1e-3, T))).astype(np.float64)
    return log_loss_np(labels, P)

# Try to load both sources if present
have_full = full_oof_p.exists() and full_te_p.exists()
have_base = base_oof_p.exists() and base_te_p.exists()

full_placeholder = True
if have_full:
    oof_logits_full = np.load(full_oof_p)
    te_logits_full  = np.load(full_te_p)
    full_placeholder = np.allclose(oof_logits_full, 0.0) or np.allclose(te_logits_full, 0.0)

if not have_base:
    raise FileNotFoundError('Baseline logits missing. Run Cell 5 to generate oof_logits.npy and test_logits.npy.')

# Load baseline artifacts and temperature
oof_logits_cpu = np.load(base_oof_p)
te_logits_cpu  = np.load(base_te_p)
T_cpu = _load_T(base_T_p)
if T_cpu is None or not np.isfinite(T_cpu):
    T_cpu = fit_temperature(oof_logits_cpu, y)
    with open(base_T_p, 'w') as f:
        json.dump({'global_T': float(T_cpu)}, f)

# Decide source with quality gate
use_full_base = have_full and (not full_placeholder)
chosen_source = 'baseline'
if use_full_base:
    T_full = _load_T(full_T_p)
    if T_full is None or not np.isfinite(T_full):
        T_full = fit_temperature(oof_logits_full, y)
        with open(full_T_p, 'w') as f:
            json.dump({'global_T': float(T_full)}, f)
    # Compute calibrated OOF losses for gate
    ll_full = _calibrated_loss(oof_logits_full, y, T_full)
    ll_cpu  = _calibrated_loss(oof_logits_cpu,  y, T_cpu)
    if ll_full + 1e-9 < ll_cpu:
        chosen_source = 'fullimg'
        oof_logits_base = oof_logits_full
        te_logits_base  = te_logits_full
        T_base = T_full
    else:
        print(f"WARNING: New full-image model is worse than baseline (full={ll_full:.6f} >= base={ll_cpu:.6f}). Falling back to baseline.")
        chosen_source = 'baseline'
        oof_logits_base = oof_logits_cpu
        te_logits_base  = te_logits_cpu
        T_base = T_cpu
else:
    if have_full and full_placeholder:
        print('AutokNN: full-image base logits are placeholders; falling back to baseline.')
    chosen_source = 'baseline'
    oof_logits_base = oof_logits_cpu
    te_logits_base  = te_logits_cpu
    T_base = T_cpu

print(f'AutokNN: Base={chosen_source}, T_base={T_base:.4f}')

oof_base_probs = row_normalize(softmax_np(oof_logits_base / max(1e-3, T_base))).astype(np.float64)
te_base_probs  = row_normalize(softmax_np(te_logits_base  / max(1e-3, T_base))).astype(np.float64)

# 2) Feature space for kNN (preference order): Multi-encoder concatenation > ConvNeXt embeddings (Cell 7) > ViT embeddings (Cells 1/2)
feat_source = 'vit'

# 2a) Multi-encoder concatenation if available — PROVENANCE-HARDENED
multi_tr_p = Path('train_embeds_multi.npy')
multi_te_p = Path('test_embeds_multi.npy')
multi_manifest_p = Path('multi_embed_manifest.json')
if (not DISABLE_MULTI) and multi_tr_p.exists() and multi_te_p.exists() and multi_manifest_p.exists():
    try:
        # Manifest must contain expected metadata and match array shapes; mtime must be >= multi arrays (created by Cell 11 this run)
        man = json.load(open(multi_manifest_p))
        man_train_shape = tuple(man.get('train_shape', []))
        man_test_shape  = tuple(man.get('test_shape', []))
        tags = man.get('tags') or man.get('encoders')
        mt_man = multi_manifest_p.stat().st_mtime
        mt_multi_ok = True
        for pth in (multi_tr_p, multi_te_p):
            if pth.exists() and pth.stat().st_mtime > mt_man + 1e-6:
                mt_multi_ok = False
                break
        if not tags:
            raise ValueError('Manifest missing tags/encoders; refusing MULTI load.')
        Xm = np.load(multi_tr_p).astype(np.float32)
        Xtm = np.load(multi_te_p).astype(np.float32)
        shape_ok = (tuple(Xm.shape) == man_train_shape) and (tuple(Xtm.shape) == man_test_shape)
        if (not mt_multi_ok) or (not shape_ok):
            raise ValueError('Manifest provenance/shape mismatch; refusing MULTI load.')
        if Xm.ndim == 2 and Xm.shape[1] > 1 and not np.allclose(Xm, 0.0) and not np.allclose(Xtm, 0.0):
            X = Xm; X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
            X_test = Xtm; X_test /= (np.linalg.norm(X_test, axis=1, keepdims=True) + 1e-12)
            feat_source = 'multi'
            print(f"AutokNN: Using MULTI-encoder embeddings (provenance-verified): train {X.shape}, test {X_test.shape}; tags={tags}")
        else:
            print('AutokNN: MULTI present but invalid content; falling back to ConvNeXt/ViT.')
    except Exception as e:
        print('AutokNN: MULTI provenance check failed; ignoring MULTI. Reason:', e)

# 2b) ConvNeXt embeddings if present and multi not used
if feat_source == 'vit':
    cn_oof_p = Path('oof_embeds_fullimg.npy')
    cn_te_p  = Path('test_embeds_fullimg.npy')
    if cn_oof_p.exists() and cn_te_p.exists():
        try:
            X_cn = np.load(cn_oof_p)
            Xt_cn = np.load(cn_te_p)
            if X_cn.ndim == 2 and X_cn.shape[1] > 1 and not np.allclose(X_cn, 0.0) and not np.allclose(Xt_cn, 0.0):
                X = X_cn.astype(np.float32); X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
                X_test = Xt_cn.astype(np.float32); X_test /= (np.linalg.norm(X_test, axis=1, keepdims=True) + 1e-12)
                feat_source = 'convnext'
                print(f'AutokNN: Using ConvNeXt embeddings: train {X.shape}, test {X_test.shape}')
        except Exception as e:
            print('AutokNN: ConvNeXt embeddings failed to load; falling back to ViT. Err:', e)

# 2c) ViT fallback
if feat_source == 'vit':
    train_embeds_raw = np.load('train_embeds.npy').astype(np.float32)
    train_ids_order = pd.read_csv('train_image_meta.csv')['id'].tolist()
    id_to_pos = {id_: i for i, id_ in enumerate(train_ids_order)}
    reindex = np.array([id_to_pos[id_] for id_ in labels_df['id']], dtype=np.int64)
    X = train_embeds_raw[reindex]
    X /= (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    Xt = np.load('test_embeds.npy').astype(np.float32)
    Xt /= (np.linalg.norm(Xt, axis=1, keepdims=True) + 1e-12)
    X_test = Xt
    print(f'AutokNN: Using ViT embeddings: train {X.shape}, test {X_test.shape}')

# 3) Precompute per-fold neighbor search (maxK) for efficient grid search
# Expanded grids per colleague guidance: larger K and finer lambda around 0.2–0.5.
K_grid = [100, 300, 500, 800, 1000, 1200, 1500, 2000]
tau_grid = [0.005, 0.01, 0.02, 0.05, 0.1]
lam_coarse = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15]
lam_fine = [round(x, 2) for x in np.linspace(0.2, 0.5, 13)]
lam_grid = sorted(set(lam_coarse + lam_fine))
K_max = max(K_grid)

# Optional reweighting knobs
CONSISTENCY_REWEIGHT_GRID = [False, True]
AGREEMENT_REWEIGHT_GRID = [False, True]  # new: weight neighbors by base posterior for their label
ALPHA_BOOST = 1.1
BETA_DISCOUNT = 0.9

fold_cache = []
for f in range(n_folds):
    tr = np.where(folds != f)[0]
    va = np.where(folds == f)[0]
    idx = faiss.IndexFlatIP(X.shape[1])
    idx.add(X[tr].astype('float32'))
    sims, idxs = idx.search(X[va].astype('float32'), K_max)
    fold_cache.append({'tr': tr, 'va': va, 'sims': sims.astype('float32'), 'idxs': idxs.astype('int32')})

def _probs_from_sims(sims_sub: np.ndarray, idxs_sub: np.ndarray, y_tr: np.ndarray, K: int, tau: float, C: int,
                     consistency_boost: bool = False, alpha: float = 1.1, beta: float = 0.9,
                     base_probs_q: np.ndarray | None = None, agreement_reweight: bool = False) -> np.ndarray:
    """
    Numerically stable neighbor weighting: softmax((sims/tau)) per query with max-subtraction.
    - consistency_boost: multiply neighbor weights by alpha if its label matches top-K majority label; else beta.
    - agreement_reweight: multiply neighbor weights by base posterior of the neighbor's label for that query.
    """
    sims_k = sims_sub[:, :K].astype('float64')
    idxs_k = idxs_sub[:, :K]
    scale = 1.0 / max(1e-8, tau)
    a = sims_k * scale
    a_max = np.max(a, axis=1, keepdims=True)
    z = a - a_max
    np.exp(z, out=z)
    z_sum = z.sum(axis=1, keepdims=True)
    z_sum[z_sum == 0.0] = 1.0
    w = z / z_sum  # (B, K)
    neighbor_labels = y_tr[idxs_k].astype(np.int64)  # (B, K)
    if consistency_boost:
        B = neighbor_labels.shape[0]
        maj = np.empty(B, dtype=np.int64)
        for i in range(B):
            maj[i] = np.bincount(neighbor_labels[i], minlength=C).argmax()
        m = np.where(neighbor_labels == maj[:, None], float(alpha), float(beta))
        w = w * m
        w_sum = w.sum(axis=1, keepdims=True)
        w_sum[w_sum == 0.0] = 1.0
        w = w / w_sum
    if agreement_reweight and base_probs_q is not None:
        # Multiply weight of each neighbor by base posterior for that neighbor's label for this query
        B, Kloc = neighbor_labels.shape
        gather = base_probs_q[np.arange(B)[:, None], neighbor_labels]
        w = w * gather
        w_sum = w.sum(axis=1, keepdims=True)
        w_sum[w_sum == 0.0] = 1.0
        w = w / w_sum
    P = np.empty((sims_k.shape[0], C), dtype=np.float64)
    P.fill(0.0)
    B, Kloc = sims_k.shape
    row_idx = np.repeat(np.arange(B, dtype=np.int64), Kloc)
    np.add.at(P, (row_idx, neighbor_labels.ravel()), w.ravel())
    P = row_normalize(P)
    return P

# 4) Grid search with temperature fit on blended OOF probs
best = {'loss': 1e9}
C = len(classes)
t0 = time.time()
for K in K_grid:
    for tau in tau_grid:
        for cboost in CONSISTENCY_REWEIGHT_GRID:
            for agree in AGREEMENT_REWEIGHT_GRID:
                oof_knn = np.zeros((X.shape[0], C), dtype=np.float64)
                for f, cache in enumerate(fold_cache):
                    tr = cache['tr']; va = cache['va']; sims = cache['sims']; idxs = cache['idxs']
                    base_va = oof_base_probs[va]
                    P_va = _probs_from_sims(sims, idxs, y[tr], K=K, tau=tau, C=C,
                                            consistency_boost=cboost, alpha=ALPHA_BOOST, beta=BETA_DISCOUNT,
                                            base_probs_q=base_va, agreement_reweight=agree)
                    oof_knn[va] = P_va
                for lam in lam_grid:
                    P_blend = row_normalize((oof_base_probs ** max(0.0, 1.0 - lam)) * (oof_knn ** lam))
                    T_b = fit_temperature_from_probs(P_blend, y)
                    P_cal = row_normalize(np.power(P_blend, 1.0 / max(1e-3, T_b)))
                    ll = log_loss_np(y, P_cal)
                    if ll < best['loss']:
                        best = {'loss': float(ll), 'K': int(K), 'tau': float(tau), 'lam': float(lam), 'T': float(T_b), 'cboost': bool(cboost), 'agree': bool(agree)}
                        print(f"AutokNN: New best -> loss={ll:.6f}, K={K}, tau={tau}, lambda={lam}, T={T_b:.4f}, cboost={cboost}, agree={agree}")

elapsed = time.time() - t0
print('AutokNN: Best config:', best)
print(f'AutokNN: Grid search elapsed {elapsed:.2f}s')

# Persist best kNN config as OUTPUT
with open('knn_blend_config.json', 'w') as f:
    json.dump({'K': best['K'], 'tau': best['tau'], 'lambda': best['lam'], 'consistency_boost': best.get('cboost', False), 'agreement_reweight': best.get('agree', False), 'search_grid': {'K': K_grid, 'tau': tau_grid, 'lambda': lam_grid}, 'base_source': chosen_source, 'feats': feat_source, 'elapsed_s': elapsed}, f)
with open('temperatures_knn_blend.json', 'w') as f:
    json.dump({'global_T': best['T'], 'base_source': chosen_source, 'feats': feat_source}, f)

# 5) Build final test predictions using best config + optional LR stacker; choose best by OOF
idx_full = faiss.IndexFlatIP(X.shape[1])
idx_full.add(X.astype('float32'))
sims_te, idxs_te = idx_full.search(X_test.astype('float32'), max(best['K'], K_max))
P_te_knn = _probs_from_sims(sims_te, idxs_te, y, K=best['K'], tau=best['tau'], C=C,
                            consistency_boost=best.get('cboost', False), alpha=ALPHA_BOOST, beta=BETA_DISCOUNT,
                            base_probs_q=te_base_probs, agreement_reweight=best.get('agree', False))
P_test_blend = row_normalize((te_base_probs ** max(0.0, 1.0 - best['lam'])) * (P_te_knn ** best['lam']))

# Test-time calibration path (MIRROR OOF): default to global temperature (best['T']); local recal disabled by default
if ENABLE_TEST_LOCAL_RECAL and sims_te.shape[1] >= 2:
    sim_gap = np.clip(sims_te[:, 0] - sims_te[:, 1], 0.0, 1.0)
    T_local = np.clip(0.2 + 2.5 * sim_gap, 0.7, 1.6)
    P_test_cal = row_normalize(np.power(P_test_blend, 1.0 / np.clip(T_local[:, None], 1e-3, None)))
else:
    P_test_cal = row_normalize(np.power(P_test_blend, 1.0 / max(1e-3, best['T'])))

# Recompute OOF kNN probs for best params (needed for potential stacker features)
oof_knn_best = np.zeros((X.shape[0], C), dtype=np.float64)
for cache in fold_cache:
    tr = cache['tr']; va = cache['va']; sims = cache['sims']; idxs = cache['idxs']
    base_va = oof_base_probs[va]
    oof_knn_best[va] = _probs_from_sims(sims, idxs, y[tr], K=best['K'], tau=best['tau'], C=C,
                                        consistency_boost=best.get('cboost', False), alpha=ALPHA_BOOST, beta=BETA_DISCOUNT,
                                        base_probs_q=base_va, agreement_reweight=best.get('agree', False))

# Optional multinomial Logistic Regression stacker (DISABLED by default due to prior leakage)
use_stacker = False
if DISABLE_STACKER:
    print('Stacker disabled by config (DISABLE_STACKER=True); proceeding with blend-only path.')
else:
    try:
        from sklearn.linear_model import LogisticRegression
        def _log_softmax_np(logits):
            m = logits.max(axis=1, keepdims=True)
            z = logits - m
            logsumexp = np.log(np.exp(z).sum(axis=1, keepdims=True) + 1e-12)
            return z - logsumexp
        z_base_oof = _log_softmax_np(oof_logits_base / max(1e-3, T_base))
        z_base_te  = _log_softmax_np(te_logits_base  / max(1e-3, T_base))
        z_knn_oof  = np.log(np.clip(oof_knn_best, 1e-8, 1.0))
        z_knn_te   = np.log(np.clip(P_te_knn,   1e-8, 1.0))
        X_oof = np.concatenate([z_base_oof, z_knn_oof], axis=1)
        X_te  = np.concatenate([z_base_te,  z_knn_te],  axis=1)
        lr = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', multi_class='multinomial', max_iter=300, n_jobs=1, verbose=0)
        lr.fit(X_oof, y)
        oof_logits_stack = lr.decision_function(X_oof).astype(np.float64)
        T_stack = fit_temperature(oof_logits_stack, y)
        P_oof_stack = row_normalize(softmax_np(oof_logits_stack / max(1e-3, T_stack)))
        ll_stack = log_loss_np(y, P_oof_stack)
        print(f'Stacker OOF (LR on [logsm(base), log(knn)]): loss={ll_stack:.6f}, T_stack={T_stack:.4f}')
        use_stacker = ll_stack + 1e-9 < best['loss']
        if use_stacker:
            te_logits_stack = lr.decision_function(X_te).astype(np.float64)
            P_test_stack = row_normalize(softmax_np(te_logits_stack / max(1e-3, T_stack)))
    except Exception as e:
        print('Stacker skipped due to error or missing sklearn:', e)
        use_stacker = False

# 5b) Hard override path (DISABLED by default to mirror OOF)
if ENABLE_HARD_OVERRIDES:
    hard_thr = 0.992
    topk = 5
    if sims_te.shape[1] >= topk:
        nn_topk_sims = sims_te[:, :topk]
        nn_topk_idx = idxs_te[:, :topk].astype(int)
        nn_labels_topk = y[nn_topk_idx]
        consensus_same = (nn_labels_topk == nn_labels_topk[:, [0]]).all(axis=1)
        min_sim_in_topk = nn_topk_sims[:, topk - 1]
        base_top1 = te_base_probs.argmax(axis=1)
        base_agree = (base_top1 == nn_labels_topk[:, 0])
        mask = (min_sim_in_topk >= hard_thr) & consensus_same & base_agree
        if mask.any():
            nn_classes = nn_labels_topk[mask, 0]
            P_sel = P_test_stack.copy() if ((not DISABLE_STACKER) and use_stacker) else P_test_cal.copy()
            P_override = np.full((mask.sum(), C), 1e-6, dtype=np.float64)
            P_override[np.arange(mask.sum()), nn_classes] = 1.0 - (C - 1) * 1e-6
            P_sel[mask] = row_normalize(P_override)
            if (not DISABLE_STACKER) and use_stacker:
                P_test_stack = P_sel
            else:
                P_test_cal = P_sel
            print(f'AutokNN: Applied top-5 consensus+base-agree overrides to {int(mask.sum())} test samples (min_sim >= {hard_thr}).')

# Choose final probabilities (blend-only if stacker disabled/not better)
final_source = 'blend'
P_final = P_test_cal
final_oof_est = best['loss']
if (not DISABLE_STACKER) and use_stacker:
    final_source = 'stacker'
    P_final = P_test_stack

sub = pd.DataFrame(P_final, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission.csv', index=False)
print(f'Final submission.csv written ({final_source}) with autonomous tuning and calibrated outputs.')
print(json.dumps({'final_loss_oof_blend_est': best['loss'], 'stacker_oof_est': (None), 'K': best['K'], 'tau': best['tau'], 'lambda': best['lam'], 'T_blend': best['T'], 'base': chosen_source, 'feats': feat_source, 'chosen': final_source, 'consistency_boost': best.get('cboost', False), 'agreement_reweight': best.get('agree', False)}, indent=2))


AutokNN: full-image base logits are placeholders; falling back to baseline.
AutokNN: Base=baseline, T_base=0.1816
AutokNN: Using ViT embeddings: train (9199, 768), test (1023, 768)


AutokNN: New best -> loss=0.694207, K=100, tau=0.005, lambda=0.01, T=1.0130, cboost=False, agree=False


AutokNN: New best -> loss=0.684906, K=100, tau=0.005, lambda=0.02, T=1.0259, cboost=False, agree=False


AutokNN: New best -> loss=0.676477, K=100, tau=0.005, lambda=0.03, T=1.0392, cboost=False, agree=False


AutokNN: New best -> loss=0.668847, K=100, tau=0.005, lambda=0.04, T=1.0527, cboost=False, agree=False


...

AutokNN: New best -> loss=0.608502, K=500, tau=0.01, lambda=0.42, T=1.1305, cboost=False, agree=False


AutokNN: New best -> loss=0.608387, K=500, tau=0.01, lambda=0.45, T=1.1431, cboost=False, agree=False


AutokNN: Best config: {'loss': 0.6083871312873045, 'K': 500, 'tau': 0.01, 'lam': 0.45, 'T': 1.1430808305740356, 'cboost': False, 'agree': False}
AutokNN: Grid search elapsed 1848.31s


Stacker disabled by config (DISABLE_STACKER=True); proceeding with blend-only path.


Final submission.csv written (blend) with autonomous tuning and calibrated outputs.
{
  "final_loss_oof_blend_est": 0.6083871312873045,
  "stacker_oof_est": null,
  "K": 500,
  "tau": 0.01,
  "lambda": 0.45,
  "T_blend": 1.1430808305740356,
  "base": "baseline",
  "feats": "vit",
  "chosen": "blend",
  "consistency_boost": false,
  "agreement_reweight": false
}


In [9]:
# Consolidated Off-box GPU Runner Generator + Handoff Verification (Authoritative)
# This cell writes gpu_runner_template.py AND prints/verifies the complete copy list.

from pathlib import Path

runner_path = Path('gpu_runner_template.py')
code = r'''#!/usr/bin/env python3
import os, json, math, time, random, argparse
from pathlib import Path
from typing import Optional, List, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image, ImageOps
import timm
from timm.loss import SoftTargetCrossEntropy
from timm.utils import ModelEmaV2
from timm.data import Mixup

SEED = 20250810
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert device == 'cuda', 'GPU not available. Please enable a GPU runtime.'

BASE = Path('.')
labels_df = pd.read_csv('labels.csv')
folds_df = pd.read_csv('fold_assignments.csv')  # must match labels_df order
test_meta = pd.read_csv('test_image_meta.csv')
sample_df = pd.read_csv('sample_submission.csv')
classes = [c for c in sample_df.columns if c != 'id']
breed_to_idx = {b:i for i,b in enumerate(classes)}
train_ids = labels_df['id'].tolist()
test_ids  = test_meta['id'].tolist()
y_all = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
folds = folds_df.set_index('id').loc[labels_df['id'], 'fold'].values.astype(int)
n_classes = len(classes)
n_folds = int(folds.max() + 1)

class DogDataset(Dataset):
    def __init__(self, ids, split: str, labels: Optional[np.ndarray] = None, img_size: int = 384, aug: bool = False):
        self.ids = ids; self.split = split; self.labels = labels
        self.root = Path('train' if split=='train' else 'test')
        if aug:
            self.tf = T.Compose([
                T.Resize(int(img_size*1.1), interpolation=T.InterpolationMode.BICUBIC),
                T.RandomResizedCrop(img_size, scale=(0.8, 1.0), interpolation=T.InterpolationMode.BICUBIC),
                T.RandomHorizontalFlip(p=0.5), T.RandAugment(num_ops=2, magnitude=9),
                T.ToTensor(), T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)), T.RandomErasing(p=0.25, value='random')
            ])
        else:
            self.tf = T.Compose([
                T.Resize(int(img_size*1.1), interpolation=T.InterpolationMode.BICUBIC),
                T.CenterCrop(img_size), T.ToTensor(), T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
            ])
    def __len__(self): return len(self.ids)
    def __getitem__(self, i):
        id_ = self.ids[i]
        img = Image.open(self.root / f"{id_}.jpg"); img = ImageOps.exif_transpose(img)
        if img.mode != 'RGB': img = img.convert('RGB')
        x = self.tf(img); img.close()
        if self.labels is None: return x, id_
        return x, self.labels[i]

def mixup_fn(num_classes: int):
    return Mixup(mixup_alpha=0.2, cutmix_alpha=0.1, prob=1.0, switch_prob=0.5, mode='batch', label_smoothing=0.05, num_classes=num_classes)

def create_model(model_name: str, num_classes: int, drop_path: float = 0.2):
    return timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_path_rate=drop_path)

def extract_feats(m: nn.Module, xb: torch.Tensor) -> torch.Tensor:
    if hasattr(m, 'forward_features'):
        feats = m.forward_features(xb)
    else:
        feats = m(xb)
    if feats.ndim == 4:
        feats = feats.mean(dim=(2,3))
    return feats

def scheduler_warm_cos(opt, epochs, iters_per_epoch, warmup_epochs=2):
    total = max(1, epochs*iters_per_epoch); warm = int(max(0, warmup_epochs)*iters_per_epoch)
    def lr_lambda(step):
        if warm > 0 and step < warm:
            return float(step+1)/float(max(1,warm))
        prog = 0.0 if total==warm else float(step-warm)/float(max(1,total-warm))
        return 0.5*(1.0+math.cos(math.pi*min(1.0,max(0.0,prog))))
    return torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)

def make_tta(scales: List[int]):
    fns=[]
    for sz in scales:
        def rf(x, s=sz):
            if x.shape[-1]==s and x.shape[-2]==s: return x
            return F.interpolate(x, size=(s,s), mode='bilinear', align_corners=False)
        fns.append(lambda x, r=rf: r(x))
        fns.append(lambda x, r=rf: torch.flip(r(x), dims=[3]))
    return fns

def fit_temperature(logits: np.ndarray, labels: np.ndarray) -> float:
    dev='cuda'
    T = torch.tensor(1.0, requires_grad=True, device=dev)
    x = torch.from_numpy(logits).to(dev)
    y = torch.from_numpy(labels).long().to(dev)
    nll = nn.CrossEntropyLoss()
    opt = torch.optim.LBFGS([T], lr=0.1, max_iter=100, line_search_fn='strong_wolfe')
    def closure():
        opt.zero_grad(); loss = nll(x/torch.clamp(T, min=1e-3), y); loss.backward(); return loss
    opt.step(closure)
    return float(T.detach().float().clamp_min(1e-3).cpu().item())

def train_fold(fold:int, model_name='convnext_base', img_size=384, epochs=8, bs=32, lr=1e-3, wd=2e-2, num_workers=8, warmup_epochs=2, tta_scales: Optional[List[int]] = None):
    tr = np.where(folds!=fold)[0]; va = np.where(folds==fold)[0]
    ids_tr = [train_ids[i] for i in tr]; ids_va = [train_ids[i] for i in va]
    y_tr = y_all[tr]; y_va = y_all[va]
    ds_tr = DogDataset(ids_tr, 'train', y_tr, img_size, aug=True)
    ds_va = DogDataset(ids_va, 'train', y_va, img_size, aug=False)
    dl_tr = DataLoader(ds_tr, batch_size=bs, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    dl_va = DataLoader(ds_va, batch_size=bs*2, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    model = create_model(model_name, n_classes, 0.2).to(device)
    ema = ModelEmaV2(model, decay=0.9998, device=device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = scheduler_warm_cos(opt, epochs, max(1,len(dl_tr)), warmup_epochs=warmup_epochs)
    mx = mixup_fn(n_classes); crit = SoftTargetCrossEntropy(); scaler = torch.cuda.amp.GradScaler()
    best_loss = 1e9; best_state=None
    for ep in range(epochs):
        model.train(); t0=time.time()
        for xb, yb in dl_tr:
            xb=xb.to(device,non_blocking=True); yb=yb.to(device,non_blocking=True)
            xb, ybm = mx(xb, yb); opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast():
                logits = model(xb); loss = crit(logits, ybm)
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update(); ema.update(model); sched.step()
        # val
        ema_m = ema.module; ema_m.eval(); vl=0.0; n=0
        with torch.no_grad():
            for xb, yb in dl_va:
                xb=xb.to(device,non_blocking=True); yb=yb.to(device,non_blocking=True)
                with torch.cuda.amp.autocast():
                    l = ema_m(xb); loss = nn.CrossEntropyLoss()(l, yb)
                vl += loss.item()*xb.size(0); n += xb.size(0)
        vl /= max(1,n)
        if vl < best_loss - 1e-6:
            best_loss = vl; best_state = {k: v.detach().cpu() for k,v in ema_m.state_dict().items()}
        print(f'Fold {fold} Epoch {ep+1}/{epochs} val_loss={vl:.5f}')
    if best_state is not None:
        ema_m.load_state_dict(best_state, strict=False)
    # OOF logits/embeds for validation indices
    oof_logits_f = np.zeros((len(va), n_classes), dtype=np.float32)
    oof_embeds_f = None
    ema_m.eval();
    with torch.no_grad():
        ptr=0
        for xb, yb in dl_va:
            xb=xb.to(device,non_blocking=True)
            l = ema_m(xb); f = extract_feats(ema_m, xb)
            lnp = l.float().cpu().numpy(); fnp = f.float().cpu().numpy()
            if oof_embeds_f is None: oof_embeds_f = np.zeros((len(va), fnp.shape[1]), dtype=np.float32)
            oof_logits_f[ptr:ptr+lnp.shape[0]] = lnp; oof_embeds_f[ptr:ptr+fnp.shape[0]] = fnp; ptr += lnp.shape[0]
    # Test-time logits/embeds with parameterized multi-scale TTA (with hflip)
    if tta_scales is None or len(tta_scales) == 0:
        # Dynamic default based on training img_size
        tta_scales = [int(img_size), int(round(img_size * 1.15))]
    tta_fns = make_tta(tta_scales)
    ds_te = DogDataset(test_ids, 'test', labels=None, img_size=img_size, aug=False)
    dl_te = DataLoader(ds_te, batch_size=bs*2, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    test_logits_f = np.zeros((len(test_ids), n_classes), dtype=np.float32)
    test_embeds_f = None
    with torch.no_grad():
        ofs=0
        for xb,_ in dl_te:
            xb=xb.to(device,non_blocking=True)
            acc_l=None; acc_f=None
            for tta in tta_fns:
                xa = tta(xb)
                la = ema_m(xa); fa = extract_feats(ema_m, xa)
                acc_l = la if acc_l is None else (acc_l+la)
                acc_f = fa if acc_f is None else (acc_f+fa)
            lnp = (acc_l / len(tta_fns)).float().cpu().numpy(); fnp = (acc_f / len(tta_fns)).float().cpu().numpy()
            nb = lnp.shape[0]
            if test_embeds_f is None: test_embeds_f = np.zeros((len(test_ids), fnp.shape[1]), dtype=np.float32)
            test_logits_f[ofs:ofs+nb] = lnp; test_embeds_f[ofs:ofs+nb] = fnp; ofs += nb
    return va, oof_logits_f, test_logits_f, float(best_loss), oof_embeds_f, test_embeds_f

def softmax_np(x):
    m = x.max(axis=1, keepdims=True); z = np.exp(x-m)
    return z / (z.sum(axis=1, keepdims=True)+1e-12)

def row_normalize(p):
    p = np.clip(p, 1e-8, 1.0); p /= p.sum(axis=1, keepdims=True); return p

def parse_scales_arg(val: Optional[str]) -> Optional[List[int]]:
    if val is None:
        return None
    s = val.strip()
    if not s:
        return None
    try:
        parts = [int(x) for x in s.replace(' ', '').split(',') if x]
        return parts if parts else None
    except Exception:
        print('WARNING: Failed to parse --tta_scales; using dynamic defaults.')
        return None

def parse_args():
    ap = argparse.ArgumentParser(description='GPU runner for Dog Breed Identification (5-fold)')
    ap.add_argument('--model_name', type=str, default='convnext_base')
    ap.add_argument('--img_size', type=int, default=384)
    ap.add_argument('--epochs', type=int, default=20)
    ap.add_argument('--bs', type=int, default=32)
    ap.add_argument('--lr', type=float, default=1e-3)
    ap.add_argument('--wd', type=float, default=2e-2)
    ap.add_argument('--num_workers', type=int, default=8)
    ap.add_argument('--warmup_epochs', type=int, default=2)
    ap.add_argument('--tta_scales', type=str, default=None, help='Comma-separated scales for TTA, e.g., "384,448,512". If omitted, defaults to [img_size, round(img_size*1.15)].')
    return ap.parse_args()

def main():
    args = parse_args()
    tta_scales = parse_scales_arg(args.tta_scales)
    cfg = {'model_name': args.model_name, 'img_size': args.img_size, 'epochs': args.epochs, 'bs': args.bs, 'lr': args.lr, 'wd': args.wd, 'num_workers': args.num_workers, 'warmup_epochs': args.warmup_epochs, 'tta_scales': (tta_scales if tta_scales is not None else [int(args.img_size), int(round(args.img_size*1.15))])}
    print('GPU runner config:', cfg)
    oof_logits = np.zeros((len(train_ids), n_classes), dtype=np.float32)
    test_logits_stack = np.zeros((n_folds, len(test_ids), n_classes), dtype=np.float32)
    oof_embeds = None; test_embeds_folds = []
    fold_losses = []
    for f in range(n_folds):
        va_idx, oof_f, te_f, vloss, oof_e_f, te_e_f = train_fold(f, model_name=args.model_name, img_size=args.img_size, epochs=args.epochs, bs=args.bs, lr=args.lr, wd=args.wd, num_workers=args.num_workers, warmup_epochs=args.warmup_epochs, tta_scales=cfg['tta_scales'])
        if oof_embeds is None: oof_embeds = np.zeros((len(train_ids), oof_e_f.shape[1]), dtype=np.float32)
        oof_logits[va_idx] = oof_f; oof_embeds[va_idx] = oof_e_f
        test_logits_stack[f] = te_f; test_embeds_folds.append(te_e_f)
        fold_losses.append(vloss)
        # Intermediate saves for robustness
        np.save('oof_logits_fullimg.npy', oof_logits)
        np.save('test_logits_fullimg.npy', test_logits_stack.mean(axis=0))
        np.save('oof_embeds_fullimg.npy', oof_embeds)
        np.save('test_embeds_fullimg.npy', np.mean(np.stack(test_embeds_folds, axis=0), axis=0))
        print(f'Fold {f} done. Best val loss={vloss:.5f}')
    with open('run_manifest_convnext_base_sz384_seed20250810.json','w') as f:
        json.dump({'seed':SEED,'device':device,'classes':n_classes,'folds':n_folds,'config':cfg,'fold_val_losses':fold_losses}, f)
    # Temperature on OOF, save manifest
    T_full = fit_temperature(oof_logits, y_all)
    with open('temperatures_fullimg.json','w') as f:
        json.dump({'global_T': float(T_full), **cfg}, f)
    print(f'Calibrated temperature (OOF): T={T_full:.4f}')
    # Optional calibrated fullimg submission (diagnostic)
    te_logits = test_logits_stack.mean(axis=0)
    P = row_normalize(softmax_np(te_logits / max(1e-3, T_full)))
    sub = pd.DataFrame(P, columns=classes); sub.insert(0,'id', test_ids)
    sub.to_csv('submission_fullimg.csv', index=False)
    # Final sanity prints
    print('Artifacts written:')
    print(' - oof_logits_fullimg.npy', oof_logits.shape, oof_logits.dtype)
    print(' - test_logits_fullimg.npy', te_logits.shape, te_logits.dtype)
    print(' - oof_embeds_fullimg.npy', oof_embeds.shape, oof_embeds.dtype)
    print(' - test_embeds_fullimg.npy', np.mean(np.stack(test_embeds_folds,0),0).shape)
    print(' - temperatures_fullimg.json',)

if __name__ == '__main__':
    main()
'''

runner_path.write_text(code)
print('Wrote gpu_runner_template.py.')

# Authoritative off-box instructions + verification
req = [
    'train',
    'test',
    'labels.csv',
    'fold_assignments.csv',
    'train_image_meta.csv',
    'test_image_meta.csv',
    'sample_submission.csv',
]
print('Required files and directories (to copy to GPU box):')
from pathlib import Path as _P
for r in req:
    p = _P(r)
    exists = p.exists()
    kind = 'DIR' if p.is_dir() else 'FILE'
    print(f' - {r} [{kind}] ->', 'OK' if exists else 'MISSING')
print('\nUsage on GPU box:')
print('1) Copy: train/, test/, labels.csv, fold_assignments.csv, train_image_meta.csv, test_image_meta.csv, sample_submission.csv, gpu_runner_template.py')
print('2) pip install -U torch torchvision timm pandas numpy pillow')
print("3) Example: python gpu_runner_template.py --model_name convnext_base --img_size 384 --epochs 20 --bs 32 --lr 1e-3 --wd 2e-2 --num_workers 8 --tta_scales 384,448")
print('   Ensure: torch.cuda.is_available() == True')
print('4) Return artifacts: oof_logits_fullimg.npy, test_logits_fullimg.npy, oof_embeds_fullimg.npy, test_embeds_fullimg.npy, temperatures_fullimg.json')
print('5) Place artifacts in notebook root and do Kernel -> Restart & Run All (Cell 8 will auto-consume).')


Wrote gpu_runner_template.py.
Required files and directories (to copy to GPU box):
 - train [FILE] -> MISSING
 - test [FILE] -> MISSING
 - labels.csv [FILE] -> OK
 - fold_assignments.csv [FILE] -> OK
 - train_image_meta.csv [FILE] -> OK
 - test_image_meta.csv [FILE] -> OK
 - sample_submission.csv [FILE] -> OK

Usage on GPU box:
1) Copy: train/, test/, labels.csv, fold_assignments.csv, train_image_meta.csv, test_image_meta.csv, sample_submission.csv, gpu_runner_template.py
2) pip install -U torch torchvision timm pandas numpy pillow
3) Example: python gpu_runner_template.py --model_name convnext_base --img_size 384 --epochs 20 --bs 32 --lr 1e-3 --wd 2e-2 --num_workers 8 --tta_scales 384,448
   Ensure: torch.cuda.is_available() == True
4) Return artifacts: oof_logits_fullimg.npy, test_logits_fullimg.npy, oof_embeds_fullimg.npy, test_embeds_fullimg.npy, temperatures_fullimg.json
5) Place artifacts in notebook root and do Kernel -> Restart & Run All (Cell 8 will auto-consume).


In [10]:
# Off-box GPU Helper: Package inputs, print checksums, and self-audit returned artifacts
# - Creates a tar.gz with required training inputs for the GPU box.
# - Prints SHA256 checksums for integrity verification.
# - If full-image artifacts are present, prints calibrated OOF losses for baseline vs full-image (quality gate preview).

import os, tarfile, hashlib, json
from pathlib import Path
import numpy as np
import pandas as pd

REQUIRED = [
    'train',
    'test',
    'labels.csv',
    'fold_assignments.csv',
    'train_image_meta.csv',
    'test_image_meta.csv',
    'sample_submission.csv',
    'gpu_runner_template.py',
]

def sha256_file(path: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(path, 'rb') as f:
        while True:
            b = f.read(chunk)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def pack_offbox_inputs(out_name='offbox_inputs.tar.gz', max_print=20):
    base = Path('.')
    missing = [p for p in REQUIRED if not Path(p).exists()]
    if missing:
        print('ERROR: Missing required paths:', missing)
        return None
    print('Creating archive:', out_name)
    with tarfile.open(out_name, 'w:gz') as tar:
        for r in REQUIRED:
            tar.add(r)
    size = Path(out_name).stat().st_size
    print(f'Archive created: {out_name} | size={size/1e6:.2f} MB')
    # Print checksums for small files only (skip train/test image trees for speed)
    small_files = [p for p in REQUIRED if Path(p).is_file()]
    for p in small_files[:max_print]:
        print(f'SHA256 {p}:', sha256_file(Path(p)))
    return out_name

def fit_temperature_np(logits: np.ndarray, y: np.ndarray) -> float:
    # Simple LBFGS on CPU via PyTorch (self-contained)
    import torch, torch.nn as nn
    device = 'cpu'
    T = torch.tensor(1.0, requires_grad=True, device=device)
    x = torch.from_numpy(logits).to(device)
    y_t = torch.from_numpy(y).long().to(device)
    nll = nn.CrossEntropyLoss()
    opt = torch.optim.LBFGS([T], lr=0.25, max_iter=100, line_search_fn='strong_wolfe')
    def closure():
        opt.zero_grad(); loss = nll(x/torch.clamp(T, min=1e-3), y_t); loss.backward(); return loss
    opt.step(closure)
    return float(T.detach().cpu().clamp_min(1e-3).item())

def softmax_np(x: np.ndarray) -> np.ndarray:
    m = x.max(axis=1, keepdims=True); z = np.exp(x-m); return z/(z.sum(axis=1, keepdims=True)+1e-12)

def row_normalize(p: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    p = np.clip(p, eps, 1.0); p /= p.sum(axis=1, keepdims=True); return p

def log_loss_np(y_true: np.ndarray, probs: np.ndarray) -> float:
    p = probs[np.arange(len(y_true)), y_true]
    return float(-(np.log(np.clip(p, 1e-15, 1.0))).mean())

def quality_gate_preview():
    # Baseline artifacts
    base_oof = Path('oof_logits.npy'); base_T = Path('temperatures.json')
    full_oof = Path('oof_logits_fullimg.npy'); full_T = Path('temperatures_fullimg.json')
    if not base_oof.exists():
        print('Baseline OOF logits missing. Run Cell 5 first.'); return
    labels_df = pd.read_csv('labels.csv'); sample_df = pd.read_csv('sample_submission.csv')
    classes = [c for c in sample_df.columns if c!='id']
    breed_to_idx = {b:i for i,b in enumerate(classes)}
    y = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
    oof_base = np.load(base_oof)
    T_base = None
    if base_T.exists():
        try:
            T_base = float(json.load(open(base_T))['global_T'])
        except Exception:
            T_base = None
    if T_base is None:
        T_base = fit_temperature_np(oof_base, y)
    ll_base = log_loss_np(y, row_normalize(softmax_np(oof_base / max(1e-3, T_base))))
    print(f'Baseline calibrated OOF loss: {ll_base:.6f} (T={T_base:.4f})')
    if not full_oof.exists():
        print('Full-image OOF logits not found yet. Bring back GPU artifacts and rerun.'); return
    oof_full = np.load(full_oof)
    if np.allclose(oof_full, 0.0):
        print('Detected placeholder full-image OOF logits (zeros). Quality gate will reject.'); return
    T_full = None
    if full_T.exists():
        try:
            T_full = float(json.load(open(full_T))['global_T'])
        except Exception:
            T_full = None
    if T_full is None:
        T_full = fit_temperature_np(oof_full, y)
    ll_full = log_loss_np(y, row_normalize(softmax_np(oof_full / max(1e-3, T_full))))
    print(f'Full-image calibrated OOF loss: {ll_full:.6f} (T={T_full:.4f})')
    if ll_full + 1e-9 < ll_base:
        print('Quality gate: PASS — full-image artifacts should be selected by Cell 8.')
    else:
        print('Quality gate: FAIL — baseline will be used by Cell 8 unless full-image improves.')

# Execute helpers
print('--- Off-box packaging ---')
pack_offbox_inputs('offbox_inputs.tar.gz')
print('\n--- Quality gate preview (if artifacts present) ---')
quality_gate_preview()
print('\nNext steps:')
print('1) Move offbox_inputs.tar.gz and extract on a GPU box.')
print('2) Run: python gpu_runner_template.py --model_name convnext_base --img_size 384 --epochs 20 --bs 32 --lr 1e-3 --wd 2e-2 --num_workers 8')
print('3) Return: oof_logits_fullimg.npy, test_logits_fullimg.npy, oof_embeds_fullimg.npy, test_embeds_fullimg.npy, temperatures_fullimg.json')
print('4) Place artifacts here and do Kernel -> Restart & Run All (Cell 8 will auto-consume via quality gate).')


--- Off-box packaging ---
ERROR: Missing required paths: ['train', 'test']

--- Quality gate preview (if artifacts present) ---
Baseline calibrated OOF loss: 0.704459 (T=0.1816)
Detected placeholder full-image OOF logits (zeros). Quality gate will reject.

Next steps:
1) Move offbox_inputs.tar.gz and extract on a GPU box.
2) Run: python gpu_runner_template.py --model_name convnext_base --img_size 384 --epochs 20 --bs 32 --lr 1e-3 --wd 2e-2 --num_workers 8
3) Return: oof_logits_fullimg.npy, test_logits_fullimg.npy, oof_embeds_fullimg.npy, test_embeds_fullimg.npy, temperatures_fullimg.json
4) Place artifacts here and do Kernel -> Restart & Run All (Cell 8 will auto-consume via quality gate).


In [11]:
# Off-box Artifact Integrator and Ensemble/Multi-Encoder Builder (No submission writes here)
# Purpose:
# - Detect one or more returned GPU artifact sets (from gpu_runner_template.py), optionally with tags.
# - Optionally ensemble multiple full-image runs by averaging logits across available sets and writing canonical names.
# - Build multi-encoder embeddings (concatenation across distinct encoders/runs) for kNN polishing in Cell 8.
# - NEVER write submission.csv here (single-source-of-truth remains Cell 8).

import os, re, json, shutil
from pathlib import Path
import numpy as np
import pandas as pd

BASE = Path('.')

# Detection rules:
# - Canonical artifact names (untagged):
#   oof_logits_fullimg.npy, test_logits_fullimg.npy, oof_embeds_fullimg.npy, test_embeds_fullimg.npy, temperatures_fullimg.json
# - Tagged variants allowed (to integrate multiple runs), e.g.: oof_logits_fullimg_convnextb_sz384.npy
#   The tag is the suffix between 'oof_logits_fullimg' and '.npy'. Matching files must share the same tag across all 5 artifacts.

def find_artifact_sets():
    files = list(BASE.glob('*'))
    names = [p.name for p in files if p.is_file()]
    # Pattern groups for tagged files
    patt = {
        'oof_logits': re.compile(r'^oof_logits_fullimg(?P<tag>[^.]*)\.npy$'),
        'test_logits': re.compile(r'^test_logits_fullimg(?P<tag>[^.]*)\.npy$'),
        'oof_embeds': re.compile(r'^oof_embeds_fullimg(?P<tag>[^.]*)\.npy$'),
        'test_embeds': re.compile(r'^test_embeds_fullimg(?P<tag>[^.]*)\.npy$'),
        'temps': re.compile(r'^temperatures_fullimg(?P<tag>[^.]*)\.json$'),
    }
    # Map tag -> available parts
    avail = {}
    for nm in names:
        for key, rgx in patt.items():
            m = rgx.match(nm)
            if m:
                tag = m.group('tag')  # '' for canonical
                d = avail.setdefault(tag, {})
                d[key] = nm
    # Keep only complete sets (all 5 present)
    complete = {tag:parts for tag, parts in avail.items() if all(k in parts for k in patt.keys())}
    return complete

def load_logits_and_embeds(parts):
    oof_l = np.load(parts['oof_logits'])
    te_l  = np.load(parts['test_logits'])
    oof_e = np.load(parts['oof_embeds'])
    te_e  = np.load(parts['test_embeds'])
    with open(parts['temps'], 'r') as f:
        t_manifest = json.load(f)
    T = float(t_manifest.get('global_T', 1.0))
    return oof_l, te_l, oof_e, te_e, T

sets = find_artifact_sets()
if not sets:
    print('No GPU artifact sets detected. Expected canonical or tagged files (see cell header).')
else:
    print(f'Detected {len(sets)} artifact set(s):', sorted(sets.keys()) if len(sets)>0 else [])
    # Filter out placeholder sets (zeros) by checking logits sum
    valid_sets = {}
    for tag, parts in sets.items():
        try:
            oof_l, te_l, oof_e, te_e, T = load_logits_and_embeds(parts)
            if np.allclose(oof_l, 0.0) or np.allclose(te_l, 0.0):
                print(f' - Skipping tag={tag!r}: placeholder zeros detected in logits.')
                continue
            valid_sets[tag] = {'parts': parts, 'oof_l': oof_l, 'te_l': te_l, 'oof_e': oof_e, 'te_e': te_e, 'T': T}
        except Exception as e:
            print(f' - Skipping tag={tag!r}: failed to load ({e}).')
    if not valid_sets:
        print('No valid (non-placeholder) artifact sets found.')
    else:
        # Ensemble logits across valid sets (simple average), write back to canonical names
        tags = sorted(valid_sets.keys())
        oof_stack = np.stack([valid_sets[t]['oof_l'] for t in tags], axis=0)
        te_stack  = np.stack([valid_sets[t]['te_l']  for t in tags], axis=0)
        oof_mean = oof_stack.mean(axis=0)
        te_mean  = te_stack.mean(axis=0)
        np.save('oof_logits_fullimg.npy', oof_mean)
        np.save('test_logits_fullimg.npy', te_mean)
        # Temperature: refit on ensembled OOF logits for robustness (CPU LBFGS)
        try:
            labels_df = pd.read_csv('labels.csv')
            sample_df = pd.read_csv('sample_submission.csv')
            classes = [c for c in sample_df.columns if c != 'id']
            breed_to_idx = {b:i for i,b in enumerate(classes)}
            y = labels_df['breed'].map(breed_to_idx).values.astype(np.int64)
            # Fit temperature (simple LBFGS on CPU)
            import torch, torch.nn as nn
            Tt = torch.tensor(1.0, requires_grad=True)
            x = torch.from_numpy(oof_mean)
            y_t = torch.from_numpy(y).long()
            nll = nn.CrossEntropyLoss()
            opt = torch.optim.LBFGS([Tt], lr=0.2, max_iter=100, line_search_fn='strong_wolfe')
            def closure():
                opt.zero_grad(); loss = nll(x/torch.clamp(Tt, min=1e-3), y_t); loss.backward(); return loss
            opt.step(closure)
            T_final = float(Tt.detach().clamp_min(1e-3).item())
        except Exception as e:
            # Fallback: mean of per-set temperatures
            T_final = float(np.mean([valid_sets[t]['T'] for t in tags]))
            print('Temperature refit failed, using mean of per-set T. Err:', e)
        with open('temperatures_fullimg.json', 'w') as f:
            json.dump({'global_T': T_final, 'ensemble_tags': tags}, f)
        print(f'Wrote ensembled full-image logits to canonical files. T={T_final:.4f} (OOF-calibrated).')

        # Build multi-encoder embeddings by concatenation across distinct sets
        # Note: We assume matching row order across sets (runner enforces id order). L2-normalization will be applied by Cell 8.
        try:
            oof_emb_list = [valid_sets[t]['oof_e'] for t in tags]
            te_emb_list  = [valid_sets[t]['te_e']  for t in tags]
            # Ensure dims align on sample axis
            n_train = oof_emb_list[0].shape[0]
            n_test  = te_emb_list[0].shape[0]
            if all(arr.shape[0]==n_train for arr in oof_emb_list) and all(arr.shape[0]==n_test for arr in te_emb_list):
                X_multi = np.concatenate(oof_emb_list, axis=1).astype(np.float32)
                Xt_multi = np.concatenate(te_emb_list,  axis=1).astype(np.float32)
                np.save('train_embeds_multi.npy', X_multi)
                np.save('test_embeds_multi.npy', Xt_multi)
                with open('multi_embed_manifest.json','w') as f:
                    json.dump({'tags': tags, 'train_shape': list(X_multi.shape), 'test_shape': list(Xt_multi.shape)}, f)
                print('Multi-encoder embeddings written: train_embeds_multi.npy, test_embeds_multi.npy')
            else:
                print('Multi-encoder concat skipped: embedding sample counts are inconsistent across sets.')
        except Exception as e:
            print('Multi-encoder concat failed:', e)

print('Integration complete. Next: Kernel -> Restart & Run All. Cell 8 will quality-gate and blend. Note: DISABLE_MULTI is currently hardcoded in Cell 8; if set True, Multi embeddings will be ignored.')


Detected 1 artifact set(s): ['']
 - Skipping tag='': placeholder zeros detected in logits.
No valid (non-placeholder) artifact sets found.
Integration complete. Next: Kernel -> Restart & Run All. Cell 8 will quality-gate and blend. Note: DISABLE_MULTI is currently hardcoded in Cell 8; if set True, Multi embeddings will be ignored.


In [12]:
# Sanitize environment per audit: remove stale multi-encoder and submission artifacts
# This cell deletes artifacts that must be regenerated deterministically in this run.
from pathlib import Path
import os

targets = [
    'train_embeds_multi.npy',
    'test_embeds_multi.npy',
    'multi_embed_manifest.json',
    'submission.csv',  # will be regenerated by Cell 8
]
removed = []
for t in targets:
    p = Path(t)
    if p.exists():
        try:
            p.unlink()
            removed.append(t)
        except Exception as e:
            print(f'WARNING: Failed to remove {t}: {e}')
print('Sanitization complete. Removed:', removed if removed else 'None (already clean).')

# Note: After this, perform Kernel -> Restart & Run All. Cell 11 will be the sole builder for multi-encoder artifacts.


Sanitization complete. Removed: ['submission.csv']


## Off-Box GPU Execution Checklist (Final Handoff)

Objective: Train strong full-image models off-box and reintegrate artifacts for a medal push.

What you already have here
- Packaged inputs: offbox_inputs.tar.gz (contains train/, test/, labels.csv, fold_assignments.csv, train/test_image_meta.csv, sample_submission.csv, gpu_runner_template.py)
- Runner script: gpu_runner_template.py (parameterized; supports --tta_scales)

On a GPU machine (≥16GB VRAM)
1) Copy and extract the package
- scp offbox_inputs.tar.gz <gpu-box>:~/work/
- tar -xzf offbox_inputs.tar.gz
- cd work/

2) Install minimal deps (Python 3.10+ recommended)
- pip install -U torch torchvision timm pandas numpy pillow

3) Run at least one strong 5-fold training (examples)
- ConvNeXt-Base @384, 20 epochs, TTA 384,448:
  python gpu_runner_template.py --model_name convnext_base --img_size 384 --epochs 20 --bs 32 --lr 1e-3 --wd 2e-2 --num_workers 8 --tta_scales 384,448
- EfficientNetV2-M @384 (optional for ensemble):
  python gpu_runner_template.py --model_name tf_efficientnetv2_m_in21ft1k --img_size 384 --epochs 20 --bs 32 --lr 1e-3 --wd 2e-2 --num_workers 8 --tta_scales 384,448
- ConvNeXt-Base @512 (optional, 12 epochs):
  python gpu_runner_template.py --model_name convnext_base --img_size 512 --epochs 12 --bs 24 --lr 1e-3 --wd 2e-2 --num_workers 8 --tta_scales 512,600

4) Return the artifacts to this notebook root
- Required per run (canonical names auto-consumed; tagged names supported for ensembling):
  - oof_logits_fullimg.npy
  - test_logits_fullimg.npy
  - oof_embeds_fullimg.npy
  - test_embeds_fullimg.npy
  - temperatures_fullimg.json
- Optional tagging for multiple runs (same tag across all five files), e.g. oof_logits_fullimg_convnextb_sz384.npy

Back in this notebook (strict procedure)
1) Kernel -> Restart & Run All (Cells 0–12)
   - Cell 11 will detect and ensemble multiple tagged sets (if any), build multi-encoder embeddings, and write canonical files.
   - Cell 8 will apply the quality/provenance gate, perform kNN blending, temperature scaling, and choose the best path.
2) IMPORTANT: Cell 12 sanitizes submission.csv. After Run-All completes, RE-RUN Cell 8 alone to regenerate submission.csv for upload.

Notes
- Quality Gate: Cell 8 only adopts full-image artifacts if their calibrated OOF log loss beats the baseline; otherwise it falls back safely.
- Single Source: Only Cell 8 writes the final submission.csv.
- Medal Path: Add 2–3 diverse models (ConvNeXt/EfficientNetV2/ViT), re-calibrate, enable multi-encoder kNN, and iterate TTA; then pseudo-label for final gains.


In [ ]:
# DEPRECATED: Emergency kNN submitter
# This cell is intentionally disabled to enforce single-source-of-truth (Cell 8 writes submission.csv).
# Per audit mandate, this cell must be inert and non-fatal during Restart & Run All.
print('Cell 14 is deprecated and disabled by policy. Single-source-of-truth: only Cell 8 may write submission.csv.')
